Originally written by Larry Chen, Summer 2022 Intern at Sector 7 under the guidance of Burak Guzelturk

# Imports

In [1]:
#!pip install line_profiler

#import pyopencl as cl
#%load_ext pyopencl.ipython_ext

In [2]:
import os
import pyFAI

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import fabio

import cv2 as cv

from skimage.registration import phase_cross_correlation
from mpl_toolkits.axes_grid1 import make_axes_locatable

from parse import parse
import glob
import json
from pyFAI.detectors import Pilatus2M
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, GridBox, Layout, ButtonStyle, VBox, HBox, Label
from ipywidgets import Text, SelectMultiple, FloatText, IntText, Accordion, GridspecLayout
import traitlets

#from ipyfilechooser import FileChooser
import concurrent.futures
import sys
import shutil
import asyncio
from watchfiles import Change, DefaultFilter
from watchfiles import awatch
from concurrent.futures import ThreadPoolExecutor
from matplotlib.ticker import MaxNLocator

import logging
import time

#from pympler import tracker

In [3]:
import warnings
warnings.filterwarnings(action='once')
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [5]:

mpl.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'axes.unicode_minus': False
    }
)
mpl.rcParams['figure.figsize'] = [8, 8]
mpl.rcParams['image.origin'] = 'lower'
import matplotlib.cm as cm

In [6]:
#%matplotlib notebook
#%matplotlib qt
%matplotlib widget

In [7]:
#logging.basicConfig(level=logging.DEBUG)
#%env PYOPENCL_COMPILER_OUTPUT=0

# Global Variables

In [51]:
#each keyword string should have '{}' either before or after a keyword, depending on where the corresponding quantity belongs
#Make sure keywords are in order

KEYWORDS_ = ['{}fshw', '{}delay']

#KEYWORDS_ = ['{}_']
#KEYWORDS_ = ['{}fshw', '{}delay', '-I_-']
#Filechoosers no longer used, may be irrelevant
#FILECHOOSER_DEFAULT_DIR = 'Z:\\diroll\\2021_04\\1p5_p25_test'

#avoid editing these, they come from KEYWORDS_, 
#(if keywords are of an undesirable format, you can replace this line with the keywords,
#ensuring the order matches KEYWORDS_)
KEYWORDS = [''.join([c for c in x if c not in '{}']) for x in KEYWORDS_]
PARTIAL_PARSE_STRING = ''.join(KEYWORDS_)
#Final string will be PREFIX+PARTIAL_PARSE_STRING+[id]+EXTENSION


UNIT = 'q_A^-1'

SAVE_DTC = True
SAVE_ORIGIN = True
SAVE_AGGREGATE = False
SAVE_COMPLETE = True

USE_SKIP = False  #for scans with periodic before time zero point ex. -5 0.1 0.2 -5 0.3 0.4 -5 ....
#(-5 X Y -5 Z A -5 B C -5) (-5 X Y...)
SKIP_PERIOD = 10
REFERENCE_PERIOD = 2

#Limited need to edit, depending on OS or convenience
SRC_DIR = 'Z:\\burakg\\2022\\06_Sharp_TRXRD_BIVO\\Ex01_Sa11'
DEST_DIR='C:\\Users\\s11idd\\Desktop\\TR_XRD_Python\\viktoria_6_2022'
MASK_DIR = 'Z:\\burakg\\2022\\06_Sharp_TRXRD_BIVO\\mask_2021_dec.tif'
PREFIX = 'Ex01_Sa11_Sc01'
EXTENSION = '.tif'

#Limited need to edit, depending on beam configuration
POLARIZATION = 0.999

#for heatmap and delay plots
USE_LOG = False
#Proportion Linear (left of log region)
LINEAR_PROPORTION = 0.5
#Area Proportion of above
LINEAR_AREA = 0.25
#limited need to edit, depending on delay linear threshold
LINEAR_THRESHOLD = 1e-9
#Heatmap Smoothness
HEATMAP_SMOOTHNESS = 500

HEATMAP_COLORMAP = 'bwr'
LINEAR_COLORMAP = 'rainbow'
REFERENCE_COLOR = 'lime'

DETECTOR_DISTANCE = 226.7414
DETECTOR_PIXEL_SIZE = 172*1e-6
DETECTOR_CENTER = [781.4048, 1641.15298]
DETECTOR_ANGLE = -0.163785
DETECTOR_ROTATION = 68.84893
DETECTOR_WAVELENGTH = 1.05969 * 1e-10
#Centering Default
CENTER_H_THRESH = 0.0
CENTER_H_ROUND = False
CENTER_H_MIN_RADIUS = 300
CENTER_H_MAX_RADIUS = 500
CENTER_H_COUNT = 3
CENTER_H_MASK = False

#Other options, such as OpenCL - > Cython if no OpenCL available
INTEGRATOR_METHOD = ('full', 'CSR', 'Cython')



# Logging Class

In [9]:
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        if len(self.out.outputs) > 200:
            self.out.outputs = self.out.outputs[:200]

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()

In [10]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

In [11]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger('__main__')
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
#logger.setLevel(logging.INFO)

# Centering Functions

## Image Preprocessing

In [12]:
def rescale(img, rescaler):
    return np.clip((rescaler*img), a_min=0, a_max=255).astype(np.uint8)

def resize(img, resizer):
    w, h = img.shape
    return cv.resize(img, (int(h*resizer), int(w*resizer)),interpolation=cv.INTER_AREA)

## Automatic Centering

In [13]:

'''
#The following parameters offer various advantages and disadvantages toward accuracy and speed
#Resizer,keep at 1 to use original image, or use less than one to trade for more speed
#Initial Threshold is used to increase the brightness of regions, leading to them being included in cv_rnd
#dp is a Hough circle parameter above 1, generally less than 2 or 3. Increasing it will trade for more speed
#param1 UpperThreshold of Hough circle canny edge. The rounding step makes this largely irrelevant, but setting it high isn't bad
if you have good contrast on the circle edges
#param2 "Threshold for center detection." and "it is the accumulator threshold for the circle 
centers at the detection stage. The smaller it is, the more false circles may be detected." - OpenCV
I can only say to not set this too high, as it starts finding unusual circles, but too low may well also be bad
#minRadius and maxRadius Obviously set these about where the circle radius you desire is, as limiting the range is a good way
to increase speed without sacrificing accuracy, unless you have a poor guess
'''
def centering_Hough(img, mask, resizer = 1, use_rounding=True, initial_threshold=0.2, circle_count = 10, 
                    dp = 1, param1 = 100, param2 = 10, minRadius = 100, maxRadius=400,
                    informative_mode = True, *args, **kwargs):
    
    if mask is not None:
        masked_image = img*(1-mask)
    else:
        masked_image = img
    
    if resizer is not 1:
        masked_image = resize(masked_image, resizer)
    
    if use_rounding:
        cv_rnd = (np.floor(masked_image/255+initial_threshold)*255).astype('uint8')
    else:
        cv_rnd = ((masked_image/255+initial_threshold)*255).astype('uint8')
    
    #try:
    circles = cv.HoughCircles(cv_rnd, cv.HOUGH_GRADIENT, dp = dp, minDist = 1e-12, 
                              param1=param1,param2=param2, 
                              minRadius=minRadius,maxRadius=maxRadius)
    '''
    except Exception as err:
        print(err)
        print('You have provided an invalid parameter.')    
    '''
    
    if circles is None:
        logger.info('Unable to find a circle.')
        
        cv_rnd = cv.cvtColor(cv_rnd, cv.COLOR_GRAY2BGR)
        cv.circle(cv_rnd,(int(cv_rnd.shape[1]/2), cv_rnd.shape[0]-60),minRadius,(0,255,0),2)
        cv.circle(cv_rnd,(int(cv_rnd.shape[1]/2), cv_rnd.shape[0]-60),maxRadius,(0,255,0),2)
        
        plt.imshow(cv_rnd, vmin= 0, vmax = 255)
        return(np.NaN, np.NaN)
    elif informative_mode:
        logger.info('A circle was found, verify the center is as desired')
        
    circles = np.uint16(np.around(circles))
    
    x_val = (1/resizer)*np.mean(circles[0,:circle_count, 0])
    y_val = (1/resizer)*np.mean(circles[0,:circle_count, 1])   
    
    if informative_mode:

        cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)        
        for i in circles[0,:circle_count]:
            # draw the outer circle
            cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv.circle(cimg,(i[0],i[1]),8,(255,0,255),-1)     
            
        logger.info('x: {}, y: {}'.format(x_val, y_val)) 
        
        #fig = plt.figure('calibration')
        #axs = fig.subplots(1,2)
        plt.close('calibration')
        fig = plt.figure('calibration')
        axs = fig.subplots(1,2)
        
        cv_rnd = cv.cvtColor(cv_rnd, cv.COLOR_GRAY2BGR)
        cv.circle(cv_rnd,(int(x_val), int(y_val)),minRadius,(0,255,0),2)
        cv.circle(cv_rnd,(int(x_val), int(y_val)),maxRadius,(0,255,0),2)
        
        axs[0].imshow(cv_rnd, vmin = 0, vmax = 255);
        axs[1].imshow(cimg);

    return (x_val, y_val)

'''
Not much tuning involved. The option to use the mask during the correlation step is available,
But I found greater success not using it, or maybe just applying it earlier
'''
def centering_autocorrelation(img, mask, mask_correlation = False, informative_mode=True, *args, **kwargs):

    
    if not mask_correlation and mask is not None:
        masked_image = img*(1-mask)
        #print('first')
    else:
        masked_image = np.copy(img)
        #print('second')
    
    h, w = masked_image.shape
    
    halved_image = (img[-w:,:w])
    if mask_correlation:
        r_mask = mask[-w:,:w]
        m_mask = np.rot90(r_mask)
        toffset = phase_cross_correlation(halved_image, np.rot90(halved_image), 
                                          reference_mask = r_mask, moving_mask = m_mask)
        offset_vector = toffset
    else:
        toffset = phase_cross_correlation(halved_image, np.rot90(halved_image))
        offset_vector = toffset[0]

    #offset_y = (offset_vector[1]+offset_vector[0])/2 #np.sqrt(2)  #r*np.cos(offset_theta)
    #offset_x = (offset_vector[1]-offset_vector[0])/2 #np.sqrt(2) #r*-np.sin(offset_theta)

    offset_x = (offset_vector[1]+offset_vector[0])/2 #np.sqrt(2)  #r*np.cos(offset_theta)
    offset_y = (offset_vector[1]-offset_vector[0])/2 #np.sqrt(2) #r*-np.sin(offset_theta)
    
    half_image_offset_x = offset_x+w/2
    half_image_offset_y = w/2+offset_y

    autocorrelation_x,autocorrelation_y = half_image_offset_x, h - half_image_offset_y
    if informative_mode:
        
        logger.info('initial offset_vector: {}'.format(offset_vector))  
        #print('theta: {}, r: {}'.format(np.rad2deg(theta), r))
        logger.info('offset x: {}, offset y: {}'.format(offset_x, offset_y))
        logger.info('halfimg x: {}, halfimg y: {}'.format(half_image_offset_x, half_image_offset_y))   
        logger.info('Final x: {},Final y: {}'.format(autocorrelation_x, autocorrelation_y))
        
        plt.close('calibration')
        fig = plt.figure('calibration')
        axs = fig.subplots(1,2)
        #fig, axs = plt.subplots(1,2)
        offset_image = np.roll(np.rot90(halved_image), int(offset_vector[1]), axis=1)
        offset_image = np.roll(offset_image, int(offset_vector[0]), axis=0)

        overlay_image = np.stack([halved_image,offset_image,np.zeros_like(halved_image)],axis=-1)
        cv.circle(overlay_image,(round(autocorrelation_x),round(w - half_image_offset_y)),20,(0,255,255),-1)

        axs[0].imshow(overlay_image, vmin= 0, vmax = 255,aspect=1);
        img_original = np.copy(img)
        cv.circle(img_original,(round(autocorrelation_x),round(autocorrelation_y)),20,(255,255,0),-1)
        axs[1].imshow(img_original, vmin= 0, vmax = 255);
        
    return (autocorrelation_x, autocorrelation_y)

<>:23: SyntaxWarning: "is not" with a literal. Did you mean "!="?


# Override default azimuthal integrator (polarization method)

In [14]:
import numexpr
from pyFAI.utils import crc32

class AzimuthalIntegratorOR(AzimuthalIntegrator):
    def __init__(self, *args, **kwargs):
        super(AzimuthalIntegratorOR, self).__init__(*args, **kwargs)


        
    #Copied from PyFAI source, but algorithm will be modified to match QXRD
    def polarization(self, shape=None, factor=None, axis_offset=0, with_checksum=False, path='numexpr'):
        #return super(AzimuthalIntegratorOR, self).polarization(shape=shape, factor=factor, axis_offset=-np.pi/4.0, with_checksum=with_checksum)
        '''
        Calculate the polarization correction accoding to the
        polarization factor:
        * If the polarization factor is None,
            the correction is not applied (returns 1)
        * If the polarization factor is 0 (circular polarization),
            the correction correspond to (1+(cos2θ)^2)/2
        * If the polarization factor is 1 (linear horizontal polarization),
            there is no correction in the vertical plane  and a node at 2th=90, chi=0
        * If the polarization factor is -1 (linear vertical polarization),
            there is no correction in the horizontal plane and a node at 2th=90, chi=90
        * If the polarization is elliptical, the polarization factor varies between -1 and +1.
        The axis_offset parameter allows correction for the misalignement of
        the polarization plane (or ellipse main axis) and the the detector's X axis.
        :param shape: the shape of the array, 
                    can be guessed most of the time from the detector definition
        :param factor: (Ih-Iv)/(Ih+Iv): varies between 0 (circular/random polarization)
                    and 1 (where division by 0 could occure at 2th=90, chi=0)
        :param axis_offset: Angle between the polarization main axis and
                            detector's X direction (in radians !!!)
        :param with_checksum: calculate also the checksum (used with OpenCL integration)
        :param path: set to numpy to enforce the use of numpy, else uses numexpr (mutithreaded)
        :return: 2D array with polarization correction (normalization) array 
                 or namedtuple if with_checksum
        '''

        shape = self.get_shape(shape)
        if shape is None:
            raise RuntimeError(("You should provide a shape if the"
                                " geometry is not yet initialized"))
        if factor is None:
            if with_checksum:
                one = numpy.ones(shape, dtype=numpy.float32)
                return pyFAI.geometry.PolarizationArray(one, crc32(one))
            else:
                return numpy.ones(shape, dtype=numpy.float32)
        elif ((factor is True) and
              (self._LAST_POLARIZATION in self._cached_array)):
            pol = self._cached_array[self._LAST_POLARIZATION]
            return pol if with_checksum else pol.array

        factor = float(factor)
        axis_offset = float(axis_offset)
        desc = pyFAI.geometry.PolarizationDescription(factor, axis_offset)
        pol = self._cached_array.get(desc)
        if pol is None or (pol.array.shape != shape):
            tth = self.twoThetaArray(shape)
            chi = self.chiArray(shape)
            factor = (factor+1.0)/2.0
            with self._sem:
                if pol is None or (pol.array.shape != shape):
                    if path == "numexpr" and numexpr:
                        pola = numexpr.evaluate(
        "1.0/(factor*(1.0 - (sin(tth)*sin(chi))**2) + (1.0-factor)*(1.0-(sin(tth)*cos(chi))**2))")

                    else:
                        #cos2_tth = numpy.cos(tth) ** 2
                        pola = 1.0/(factor*(1.0 - (np.sin(tth)*np.sin(chi))**2) + (1.0-factor)*(1.0-(np.sin(tth)*np.cos(chi))**2))
                    pola = pola.astype(np.float32)
                    polc = crc32(pola)
                    pol = pyFAI.geometry.PolarizationArray(pola, polc)
                    self._cached_array[desc] = pol
        self._cached_array[self._LAST_POLARIZATION] = pol
        return pol if with_checksum else pol.array        

pyfai"0.5 * (1.0 + cos(tth)**2 - factor * cos(2.0 * (chi + axis_offset)) * (1.0 - cos(tth)**2))"

qxrd"factor*(1.0 - (sin(tth)*sin(chi))**2) + (1.0-factor)*(1.0-(sin(tth)*cos(chi))**2)"

aior = AzimuthalIntegratorOR(detector='Pilatus2M')

# DataScan Class

In [60]:

#Class for data management

#Pandas dataframe (labels, i_y, centers)
#i_x
#params for centering algorithm{rescaler,}
#detector parameters
#Integration params: integration parameters
#general params for  plus directories/prefixes

class DataScanManager:
    
    def __init__(self):
        
        #Data (for plots and related applications) - HD5
        self.labeled_data = pd.DataFrame()
        self.i_x = pd.DataFrame()
        
        #File+Image Processing - JSON data?
        self.center_params = None
        self.detector_params = None
        self.integration_params = None
        self.general_params = None
        
        #Constructed during runtime
        self.AI = None
        self.mask = None
        self.centering_alg = None
        self.center_alg_params = None
        self.calculation_params = {'track_center': False, 'norm_range': [0,1]}
        self.lock = asyncio.Lock()
        self.executor = ThreadPoolExecutor()
        #self.callbacks = {}
        #self.normalization_range = [0, 1]
        #self.track_center = False

    def savedata(self, name=None):
        
        dest_dir = self.general_params['dest_dir']
        
        if name is None:
            name = self.general_params['prefix']
            
        os.makedirs(os.path.dirname(os.path.join(dest_dir, name)), exist_ok=True)    
        json_name = os.path.join(dest_dir, name + '_parameters.json')
        
        json_dict = {
                     'general': self.general_params,
                     'center': self.center_params,
                     'detector': self.detector_params,
                     'integration': self.integration_params,
                     'calculation': self.calculation_params
                    }

        with open(json_name, 'w') as outfile:
            json.dump(json_dict, outfile, indent=4)

        #follow with saving to hd5
        async def save_hdf5():
            async with self.lock:
                self.labeled_data.to_hdf(os.path.join(dest_dir,name+'_data.h5'), key='labeled data')
                self.i_x.to_hdf(os.path.join(dest_dir,name+'_data.h5'), key='i_x')
                #if SAVE_COMPLETE:
                self.labeled_data.to_csv(os.path.join(dest_dir,name+'_complete.csv'))  
                    
        loop = asyncio.get_event_loop()
        self.save_task = loop.create_task(save_hdf5())        
        #executor = ThreadPoolExecutor()
        #asyncio.get_event_loop().run_in_executor(executor = executor, func = save_hdf5)
        
    #Add existence checks?
    def loaddata(self, name=None, suffix='data'):

        if name is None:
            name = os.path.join(self.general_params['dest_dir'],self.general_params['prefix'])
            
        json_name = os.path.join(self.general_params['dest_dir'], name + '_parameters.json')
        
        #follow with loading hd5
        dest_dir = self.general_params['dest_dir']
        self.labeled_data = pd.read_hdf(os.path.join(dest_dir,name+'_data.h5'), key='labeled data')
        self.i_x = pd.read_hdf(os.path.join(dest_dir,name+'_data.h5'), key='i_x')          
        
        with open(json_name, 'r') as infile:
            json_dict = json.load(infile)
            
            params = {'center':self.center_params,
                      'detector':self.detector_params,
                      'integration':self.integration_params,
                      'general':self.general_params,
                      'calculation':self.calculation_params}
            
            for key in json_dict.keys(): 
                params[key].update(json_dict[key])
            '''
            self.center_params = json_dict['center']
            self.detector_params = json_dict['detector']
            self.integration_params = json_dict['integration']
            self.general_params = json_dict['general']
            self.calculation_params = json_dict['calculation']
            '''
    #Utility Functions
    '''
    src_dir: where images are stored, ideally an absolute path, but is left to user to determine if a relative path is preferred
    dest_dir: where the processed data will be saved should this object be saved
    prefix: A prefix on all files in the directory which should be included
    keywords: name of each quantity in the filename given in order they occur
    extension: file extension
    separator: separator between each keyword and prefix
    '''
    def assign_general_params(self, src_dir, dest_dir, prefix, scan_name='Sc01', keywords=KEYWORDS, extension = '.tif', mask_dir = None):
        self.general_params = {
                               'src_dir': src_dir,
                               'dest_dir': dest_dir,
                               'prefix' : prefix,
                               'keywords': keywords,
                               'extension': extension,
                               'scan_name': scan_name,
                               'mask_dir': mask_dir
                              }
        

    def assign_detector_params(self, distance, pixel_size, center, tilt_angle, tilt_rotation, wavelength, detector='Pilatus2M'):
         self.detector_params = {
                                 'distance': distance,
                                 'pixel_size': pixel_size,
                                 'center' : center,
                                 'tilt_angle': tilt_angle,
                                 'tilt_rotation': tilt_rotation,
                                 'wavelength': wavelength,
                                 'detector' : detector
                                }
        
    
    def assign_center_params(self, method, parameters=None):
        assert(type(method) is str)
        
        self.center_params = {'method': method, 'rescaler': 255/1000}
        
        if method is 'hough':
            #self.centering_alg = centering_Hough
            self.center_params.update({
                                       'minRadius': 200,
                                       'maxRadius': 400,
                                       'dp': 1,
                                       'resizer': 1, 
                                       'use_rounding': True,
                                       'initial_threshold': 0.2,
                                       'circle_count': 10,
                                       'param1': 100,
                                       'param2': 10
                                      })            
        elif method is 'correlate':
            #self.centering_alg = centering_auotocorrelation
            self.center_params.update({
                                       'mask_correlation': False
                                      })       
        else:
            print('method: ', method, 'not known')
        
        #For all changes, override defaults
        if type(parameters) is dict:
            self.center_params.update(parameters)
        
        self.update_center_alg_params()
        
    
    def assign_integration_params(self, npt=1000, normalization=1.0, radial_range=None, azimuth_range=None, method='csr'):
        self.integration_params = {
                                   'npt': npt,
                                   'normalization_factor': normalization,
                                   'radial_range': radial_range,
                                   'azimuth_range': azimuth_range,
                                   'method': method
                                  }
    #utility function
    def update_center_alg_params(self):
        params = self.center_params.copy()
        method = params.pop('method')
        params.pop('rescaler', None)
        centering_algs = {
                          'hough': centering_Hough,
                          'correlate': centering_autocorrelation
                         }
        if method is 'hough':
            params.pop('mask_correlation', None)
        elif method is 'correlate':
            params = {'mask_correlation': params['mask_correlation']}
        else:
            print('method provided is not a centering method available')
        self.centering_alg = centering_algs[method]
        self.center_alg_params = params
        
    def get_center(self, img, rescaler=None, informative_mode = False):
        if rescaler is None:
            rescaler = self.center_params['rescaler']
        
        img = rescale(img, rescaler)
        params = self.center_alg_params
        
        center = self.centering_alg(img, self.mask, **params, informative_mode=informative_mode)
        return center
    #Preparation Functions
    def setup_AI(self):
        
        if not self.integration_params:
            print('Please assign integration parameters before setting up azimuthal integrator.')
            return
        detector = self.detector_params['detector']
        wavelength = self.detector_params['wavelength']
        center = self.detector_params['center']
        pixel_size = self.detector_params['pixel_size']
        distance = self.detector_params['distance']
        tilt_angle = self.detector_params['tilt_angle']
        tilt_rotation = self.detector_params['tilt_rotation']
        detector_ = pyFAI.detector_factory(detector)
        detector_.set_pixel1(pixel_size)
        detector_.set_pixel2(pixel_size)
        
        #ai = AzimuthalIntegrator(detector=detector_)
        ai = AzimuthalIntegratorOR(detector=detector_)
        
        ai.set_wavelength(wavelength)
        ai.setFit2D(distance, center[0], center[1],
                    tilt=tilt_angle,tiltPlanRotation=tilt_rotation)
        self.AI = ai
        return

    def get_current_files(self):
        src_dir = self.general_params['src_dir']
        prefix = self.general_params['prefix']
        extension = self.general_params['extension']
        
        files = glob.glob(os.path.join(src_dir, prefix+'*'+extension))
        filenames = [os.path.basename(x) for x in files]
        return filenames
    
    def preload_mask(self, file=None, transform=(lambda x: 1-x)):
        if file is None:
            mask_dir = self.general_params['mask_dir']
            if mask_dir is None:
                print('Neither file string nor generalparams[mask_dir] was provided, please provide a source for mask file.')
                return
            file = mask_dir
        mask_ = fabio.open(file)
        self.mask = transform(mask_.data)
        mask_.close()
    
    #Calibration Functions
    def calibrate_circle(self, target = None, rescaler = 255/10000, autoassign=True):
        
        if target is None:
            src_dir = self.general_params['src_dir']
            prefix = self.general_params['prefix']
            extension = self.general_params['extension']
            files = glob.glob(os.path.join(src_dir, prefix+'*'+extension))
            file = files[0]
        else:
            file = target
        
        img_ = fabio.open(file)
        img = np.copy(img_.data)
        img_.close()       
        #print(img.shape)
        self.update_center_alg_params()
        
        center = self.get_center(img, rescaler=rescaler, informative_mode = True)
        if (autoassign) and (center[0] is not np.NaN):
            self.detector_params['center'] = [center[0].astype(float), center[1].astype(float)]
            #print('Center has been assigned to detector parameters.')
            self.center_params['rescaler'] = rescaler
            #print('Centering rescaler has been updated.')
        return center
            
    #To account for the case the target isn't in the same directory
    def calibrate_normalization(self, target = None, mean_region = None, autoassign=True):
        '''
        if self.AI is None:
            print('The Azimuthal Integrator has not been setup yet')
            return
        '''
        self.setup_AI()
        self.preload_mask()
        
        if target is None:
            src_dir = self.general_params['src_dir']
            prefix = self.general_params['prefix']
            extension = self.general_params['extension']
            files = glob.glob(os.path.join(src_dir, prefix+'*'+extension))
            file = files[0]
        else:
            file = target
        
        if not mean_region:        
            mean_region = self.calculation_params['norm_range']
        
        image_ = fabio.open(file)
        image = np.copy(image_.data)
        image_.close()
        
        integ_no_norm_params = self.integration_params.copy()
        #integ_no_norm_params.pop('normalization_factor')
        
        integration_result = self.AI.integrate1d_ng(image, mask = self.mask, polarization_factor=POLARIZATION, correctSolidAngle=True, unit = UNIT, 
                                                    safe=True, **integ_no_norm_params)
        
        plt.close('calibration')
        fig = plt.figure('calibration')
        ax = fig.subplots(1,1)        
        
        ax.plot(integration_result[0], integration_result[1])
        
        ax.axvspan(mean_region[0], mean_region[1], color='red', alpha=0.5)
        
        region_indices = np.where((integration_result[0]>=mean_region[0]) & (integration_result[0]<mean_region[1]))
        
        mean_val = np.mean((integration_result[1])[region_indices])
        if autoassign:
            self.integration_params['normalization_factor'] = mean_val.astype(float)
        return mean_val    
    
    async def update_dataframe(self, new_dataframe, i_x):
        async with self.lock:
            self.labeled_data = new_dataframe
            self.i_x = i_x

    async def dataviewer_pull(self):
        val = {}
        async with self.lock:
            val.update({'labeled_data':self.labeled_data.copy(), 
                        'keywords':self.general_params['keywords'].copy(),
                        'i_x':self.i_x.copy()})
        return val
    
    def normalize_integ(self, integration_result):
        norm_range = self.calculation_params['norm_range']
        region_indices = np.where((integration_result[0]>=norm_range[0]) & (integration_result[0]<norm_range[1]))
        mean_val = np.mean((integration_result[1])[region_indices])
        return (integration_result[0], integration_result[1]/mean_val)
    
    #Integration Functions
    #Integration for a single image
    def integ_func(self, filename, track_center=False, use_normalization = True):
        
        image_ = fabio.open(filename)
        image = np.copy(image_.data)
        image_.close()
            
        integ_res = self.AI.integrate1d_ng(image, mask = self.mask, polarization_factor = POLARIZATION, correctSolidAngle=True, unit=UNIT,
                                           **self.integration_params)
        
        ix = integ_res[0]
        iy = integ_res[1]
        
        if use_normalization:    
            ix, iy = self.normalize_integ(integ_res)
        
        if track_center:
            cx, cy = self.get_center(image, informative_mode = False)
            return ix, iy, cx, cy
        else:
            return ix, iy
    
    async def static_integrate(self, track_center = None):
        
        self.setup_AI()
        self.preload_mask()
        
        await self.update_dataframe(pd.DataFrame(), None)
        
        tasks = []
        loop = asyncio.get_event_loop()            
        executor = self.executor
        files = self.get_current_files()
        
        for file in files:
            tasks.append(loop.run_in_executor(executor, self.static_calculate_file, file, track_center))
        total_len = len(files)

        while tasks:
            logger.info('task_length: {}'.format(len(tasks)))
            finished, remaining = await asyncio.wait(tasks, timeout=2)

            batch = []
            i_x = None
            new_ld = pd.DataFrame()
            for elem in finished:

                res = elem.result()
                fut = res[0]
                i_x = res[1]
                batch.append(fut)
            
            if len(batch) > 0:
                new_ld = pd.concat(batch)
                async with self.lock:
                    new_ld = pd.concat([self.labeled_data, new_ld]).drop_duplicates([('Common', 'filename')], keep='last')
                await self.update_dataframe(new_ld, i_x)
            tasks = remaining
            if tasks:
                logger.info('Progress: {} files out of {}'.format(total_len-len(tasks), total_len))
            else:
                logger.info('Static Integrate Complete')
            
    def static_calculate_file(self, filename, track_center = None):
        if track_center == None:    
            track_center = self.calculation_params['track_center']
        src_dir = self.general_params['src_dir']
        keywords = self.general_params['keywords']
        prefix = self.general_params['prefix']
        extension = self.general_params['extension']
        scan_name = self.general_params['scan_name']
            
        if track_center:
            self.update_center_alg_params()
        
        #print(filename)
        #parse_string = (''.join([keyword+"{}" for keyword in [prefix]+keywords]))+extension
        parse_string = prefix+PARTIAL_PARSE_STRING+'{}'+extension
    
        df = pd.DataFrame(columns=['filename'])
        df['filename']=[filename]

        integ_res = pd.DataFrame(df['filename'].apply(lambda x: self.integ_func(os.path.join(src_dir, x), track_center)))

        c_vals = [[filename]+list(parse(parse_string, filename))]    
        if track_center:
            df[['i_x', 'i_y', 'c_x', 'c_y']] = pd.DataFrame(integ_res['filename'].to_list(), columns=['i_x', 'i_y', 'c_x', 'c_y'])
            c_vals = np.concatenate((c_vals, df[['c_x', 'c_y']].values),axis=1)
            df_c = pd.DataFrame(c_vals, index=pd.Index([scan_name],name='Scan'), columns = ['filename']+keywords+['id']+['c_x', 'c_y'])
        else:
            df[['i_x', 'i_y']] = pd.DataFrame(integ_res['filename'].to_list(), columns=['i_x', 'i_y'])
            df_c = pd.DataFrame(c_vals, index=pd.Index([scan_name],name='Scan'), columns = ['filename']+keywords+['id'])

        i_x = pd.Series(df['i_x'].values[0], name='i_x')
        df_i = pd.DataFrame(np.expand_dims(df['i_y'][0], axis=0), columns = pd.MultiIndex.from_product([['IFull'], df['i_x'][0]]))

        type_dict = {'filename':object, 'id':int}
        for key in keywords:
            type_dict[key] = float
        df_c = df_c.astype(type_dict)
        df_c = df_c.set_index(['id']+keywords, append=True)
        df_c = pd.concat({'Common':df_c}, axis=1)

        df_i.index = df_c.index
        df = pd.concat([df_c, df_i], axis=1)

        return df, i_x
        
    async def live_calculate(self, filenames, track_center = None):
        df = pd.DataFrame()
        i_x = None
        tasks = []
        loop = asyncio.get_event_loop()            
        executor = self.executor
        total_len = len(filenames)
        
        for file in filenames:
            tasks.append(loop.run_in_executor(executor, self.static_calculate_file, file, track_center))
            
        while tasks:
            logger.info('live batch length: {}'.format(len(tasks)))
            finished, remaining = await asyncio.wait(tasks, timeout=2)

            batch = []
            i_x = None
            
            new_ld = pd.DataFrame()
            
            for elem in finished:
                
                res = elem.result()
                fut = res[0]
                i_x = res[1]
                batch.append(fut)
            
            if len(batch) > 0:
                new_ld = pd.concat(batch)
                async with self.lock:
                    new_ld = pd.concat([self.labeled_data, new_ld]).drop_duplicates([('Common', 'filename')], keep='last')
                await self.update_dataframe(new_ld, i_x)
            tasks = remaining
            if tasks:
                logger.info('Live: {} files out of {}'.format(total_len-len(tasks), total_len))
            else:
                logger.info('current live batch done')
            
            #df, i_x = await self.static_calculate_file(filename, track_center = track_center)
        #else:
        #print("Live File Update: {} files".format(len(filenames)))
        #await self.update_dataframe(df, i_x)
        #with self.lock:
        #    self.labeled_data = 

<>:139: SyntaxWarning: "is" with a literal. Did you mean "=="?


# Data Manipulation Class

Groups and Aggregates data 

deltaI: Plot residual/difference plots of intensity over q, plotting per delay/other parameters \
Heatplot: delay on x axis, q values on y, intensity on color axis \
Delay: delay on x axis, intensity on y axis, plot per q

In [16]:
class DataViewer3:
    
    #construct
    def __init__(self, dsm):
        self.dsm = dsm
        self.group_categories = None
        self.name_dict = {}
        self.mean_diff_i = None
        self.aggregation_params = {'groups': [], 'ref_grp':None, 'aggregator': 'avg', 'n_range':[0, 1]}
        self.reference = None
        self.properties = None
        
    async def pull_data(self):
        
        resource = await self.dsm.dataviewer_pull()
        keywords = resource['keywords']
        i_x = resource['i_x']
        labeled_data = resource['labeled_data'].sort_index()
        labeled_data = labeled_data.reset_index('delay')
        labeled_data['delay'] = -1*labeled_data['delay']
        labeled_data.set_index(['delay'], inplace=True, append=True)
        self.load_assister(labeled_data, keywords, i_x)
    
    def load_assister(self, labeled_data, keywords, i_x):
        
        self.keywords = keywords
        self.i_x = i_x
        self.properties = labeled_data
        self.linear_labels = self.properties.columns.levels[0][1:].values
    
    def loaddata(self, data_dir, prefix):
            
        json_name = prefix + '_parameters.json'
        general_params = {}
        infile = os.path.join(data_dir, json_name)
        
        with open(json_name, 'r') as infile:
            json_dict = json.load(infile)
            general_params.update(json_dict['general'])
        
        keywords = general_params['keywords']
                
        name = os.path.join(self.general_params['dest_dir'],self.general_params['prefix'])        
        
        #follow with loading hd5
        dest_dir = general_params['dest_dir']
        labeled_data = pd.read_hdf(os.path.join(dest_dir,name+'_data.h5'), key='labeled data')
        
        labeled_data = labeled_data.reset_index('delay')
        labeled_data['delay'] = -1*labeled_data['delay']
        labeled_data.set_index(['delay'], inplace=True, append=True)
        
        i_x = pd.read_hdf(os.path.join(dest_dir,name+'_data.h5'), key='i_x') 
        
        self.load_assister(labeled_data, keywords, i_x)

    def savedata(self, name=None):
        
        if name is None:
            name = self.dsm.general_params['prefix']
        
        dest_dir = self.dsm.general_params['dest_dir']
        os.makedirs(os.path.dirname(os.path.join(dest_dir, name)), exist_ok=True)
        
        if SAVE_COMPLETE:
            self.properties.to_csv(os.path.join(dest_dir,name+'_complete.csv'))
        
        if SAVE_AGGREGATE:
            self.get_intensity(key=None).to_csv(os.path.join(dest_dir,name+'_aggregate.csv'))
        
        if SAVE_ORIGIN:
            mean_val = self.get_intensity(key=None).copy()
            mean_val.insert(0, 'units', 'intensity')
            mean_val = mean_val.set_index('units', append=True).T
            mean_val.insert(0, 'Reference', self.reference) 
            mean_val.index.rename(['ROI', UNIT], inplace=True)
            mean_val.to_csv(os.path.join(dest_dir, name+'_origin.csv'))
        
        if SAVE_DTC:
            test_dtc = self.get_intensity(key=None).T

            test_dtc = self.mean_diff_i.copy().T
            test_dtc = test_dtc.droplevel(0).T.droplevel(0).reset_index('delay').T.reset_index(0)
            test_dtc.iloc[0,0]=0        
            test_dtc = test_dtc.to_numpy()

            np.savetxt(os.path.join(dest_dir,name+'.dtc'), test_dtc, fmt='% e', delimiter='  ')        
        
    def group_mode(self, categories = None):
        #TODO: check that all categories are in keywords
        if categories != None:     
            self.group_categories = categories
        self.grouped_properties = self.properties.groupby(self.group_categories, as_index=True)
        self.group_names = [f[0] for f in self.grouped_properties]
        self.name_dict = {}
        for name in self.group_names:
            name_str = self.group_tostring(name)
            self.name_dict[name_str] = name

    def do_skip_difference(self):
        ref_grps = self.aggregation_params['ref_grp']
        ids = self.properties.reset_index(level=[0, 1])['id']
        true_min = ids.min()
        first_ref = ids.loc[ref_grps].min()
        
        ref_id = ids-first_ref
        grps = self.aggregation_params['groups']
        max_len = SKIP_PERIOD*len(ids.loc[ref_grps]) #['id']-first_ref
        
        ids = (((ref_id // SKIP_PERIOD)*SKIP_PERIOD+REFERENCE_PERIOD*((ref_id % SKIP_PERIOD)/REFERENCE_PERIOD).round()).astype(int))#.loc[0]
        #ids = ids[ids<len(xg.dv.properties)]
        ids[ids<0] = 0
        ids[ids>=max_len] = ids[ids<max_len].max()
        
        
        all_ref = self.properties.iloc[ids+(first_ref-true_min)]
        
        difference = self.properties.drop(columns='Common').subtract(all_ref.drop(columns='Common').values)
        return difference
    
    def do_aggregation(self):
            
        if self.aggregation_params['aggregator'] == 'avg':
            self.mean_difference()
        elif self.aggregation_params['aggregator'] == 'nth':
            self.get_nth()
        else:
            print('Not a valid aggregation method')
            
    def mean_difference(self):
        ref_grps = self.aggregation_params['ref_grp']
        #for label in self.linear_labels:
        means = self.grouped_properties.mean() #.apply(lambda x: intensity[x].mean(axis=1))).unstack() #.set_index(self.group_categories)
        self.mean_diff_i = means
        if ref_grps and len(ref_grps) > 0:

            reference = means.loc[ref_grps]
            reference = reference.mean(axis = 0)
            self.reference = reference
            if USE_SKIP:
                self.mean_diff_i = self.do_skip_difference()
                self.mean_diff_i = self.mean_diff_i.groupby(self.group_categories).mean() #.size().min()
            else:
                self.mean_diff_i = means.subtract(reference)
                
    def get_nth(self):

        ref_grps = self.aggregation_params['ref_grp']
        n_range = self.aggregation_params['n_range']
        n_vals = self.grouped_properties.nth(list(range(n_range[0], n_range[1]))).drop(columns='Common')
        self.mean_diff_i = n_vals.groupby(self.group_categories).mean()

        if ref_grps:
            reference = n_vals.loc[ref_grps]
            reference = reference.mean(axis=0)
            self.reference = reference
            if USE_SKIP:
                self.mean_diff_i = self.do_skip_difference() 
                self.mean_diff_i = self.mean_diff_i.groupby(self.group_categories).nth(list(range(n_range[0], n_range[1]))).drop(columns='Common') #.size().min()
                self.mean_diff_i = self.mean_diff_i.groupby(self.group_categories).mean()
            else:               
                self.mean_diff_i = self.mean_diff_i.subtract(reference) 
     
    
    #Note, aggregation_params['groups'] is empty when [], but takes all if None
    def get_intensity(self, key=None):
        grps = self.aggregation_params['groups']
        intensities = self.mean_diff_i
        if key:
            intensities = intensities[key]
        if grps != None:
            intensities = intensities.loc[grps]
            
        return intensities
    
    def save_mean_diff(self, name=None, dest_dir='.'):
        if name == None:
            name ='mean_diff.h5'
        
        for label in self.linear_labels:
            intensities = self.get_intensity(label)
            intensities.to_hdf(os.path.join(dest_dir, name), key=label)
    
    def get_group_names(self):
        #return [group_tostring(grp) for grp in self.group_names]
        return list(self.name_dict.keys())
    def parse_names_string(self, names):
        if (type(names) == list) or (type(names) == tuple):
            return [self.name_dict[name] for name in list(names)]
        else:
            return self.name_dict[names]
    def group_tostring(self, group):
        if type(group) is str:
            return group
        elif len(self.group_categories) == 1:
            return "{}: {}".format(self.group_categories[0], group)
        else:
            res = []
            for i, val in enumerate(self.group_categories):
                res.append("{}: {}".format(val, group[i]))
            string = ','.join(res)
            return string

# Plotting Class with Plot Memory

In [20]:
#Fully detach plotting
class PlotManager:
    
    def __init__(self, dataviewer):
        self.DV = dataviewer
        self.group_params = {'deltaI_dict': {}, 'delay_dict': {}, 'deltaI_choice':[], 'delay_choice':[]}
        self.plot_params = {'v_range': .12, 'q_index': [0, 100], 'step': 5, 'sample': None, 'center': False, 
                            'use_q': False, 'use_group': False}
        self.axs = None
        self.fig = None
        self.output = pd.DataFrame()
    
    def recreate_subplots(self, rows, columns, output=None):
        #self.fig, self.axs = plt.subplots(rows, columns)
        #if type(self.axs) != list:
        #    self.axs = [self.axs]
        plt.close(self.fig) #.clear()
        self.fig = plt.figure('Interactive Plot')
        self.axs = self.fig.subplots(rows, columns)
            
        if type(self.axs) != list:
            self.axs = [self.axs]
        
    def plot_heatmap(self, key='IFull', ax_i = 0):
        v_range = self.plot_params['v_range']

        plt.close(self.fig)
        self.fig = plt.figure('Interactive Plot')
        
        #Stores plot data
        self.output = pd.DataFrame()

        base = self.DV.aggregation_params['ref_grp']    
        temp = self.DV.get_intensity(key)
        ref = self.DV.reference        
        i_x = self.DV.i_x
        delays = (temp.index.to_frame(index=False)['delay']).values
        ys = range(len(temp.columns))    
        
        val = self.DV.get_intensity('IFull').reset_index(['delay'])   

        if USE_LOG:
            self.axs = self.fig.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios':[1,1/LINEAR_AREA-1]})

            ax = self.axs[0]
            ax2 = self.axs[1]             
            linear_max = int(np.ceil(LINEAR_PROPORTION*len(delays)))
            max_delay = np.max(delays[:linear_max])
            
            ax.set_xlim(np.min(delays), np.max(delays[:linear_max]))
            
            ax2.set_xlim(np.min(delays[linear_max:]), np.max(delays))
            ax2.set_xscale('log')     # heatmap plot selector for lin vs. log  (log or linear)            

            Z1 = (val[val['delay'] <= max_delay].set_index(['delay'], append=True).T).values    
            X1, Y1 = np.meshgrid(delays[:linear_max], i_x)
            cont1 = ax.contourf(X1, Y1, Z1, HEATMAP_SMOOTHNESS, cmap = HEATMAP_COLORMAP, vmin = -v_range, vmax = v_range)

            Z2 = (val[val['delay'] > max_delay].set_index(['delay'], append=True).T).values    
            X2, Y2 = np.meshgrid(delays[linear_max:], i_x)
            cont2 = ax2.contourf(X2, Y2, Z2, HEATMAP_SMOOTHNESS, cmap = HEATMAP_COLORMAP, vmin = -v_range, vmax = v_range)
            
            ax.spines['right'].set_visible(False)

            ax.yaxis.set_ticks_position('left')
            ax.xaxis.set_ticks_position('bottom')    
            ax2.yaxis.set_ticks_position('right')
            ax2.xaxis.set_ticks_position('top')
            self.fig.subplots_adjust(wspace=0.02)  
            
            axr = ax.twiny()
            axr.plot(ref.values, i_x, c=REFERENCE_COLOR)
            axr.axis('off')

            divider = make_axes_locatable(ax2)
            cax = divider.append_axes("right", "5%", pad="10%")
            cbar = mpl.colorbar.ColorbarBase(cax, cmap=cont2.cmap, norm=cont2.norm)
            cbar.set_ticks([*np.linspace(-v_range, v_range, 11)])
            
            self.output = pd.DataFrame([pd.Series(X1.flatten(), name='X1'), pd.Series(Y1.flatten(), name='Y1'), pd.Series(Z1.flatten(), name='Z1'), 
                                        pd.Series(X2.flatten(), name='X2'), pd.Series(Y2.flatten(), name='X2'), pd.Series(Z2.flatten(), name='Z2')])
            self.output = self.output.T
        else:
            self.axs = self.fig.subplots(1, 1, squeeze=False)[0]            
            ax = self.axs[0]       
            ax.set_xlim(np.min(delays), np.max(delays))         

            Z = (val.set_index(['delay'], append=True).T).values    
            X, Y = np.meshgrid(delays, i_x)
            cont = ax.contourf(X, Y, Z, HEATMAP_SMOOTHNESS, cmap = HEATMAP_COLORMAP, vmin = -v_range, vmax = v_range)
            
            axr = ax.twiny()
            axr.plot(ref.values, i_x, c=REFERENCE_COLOR)
            axr.axis('off')
            
            cax, _ = mpl.colorbar.make_axes(ax)
            cbar = mpl.colorbar.ColorbarBase(cax, cmap=cont.cmap, norm=cont.norm)
            cbar.set_ticks([*np.linspace(-v_range, v_range, 11)])
            self.output = pd.DataFrame([pd.Series(X.flatten(), name='X'), pd.Series(Y.flatten(), name='Y'), 
                                        pd.Series(Z.flatten(), name='Z')])
            self.output = self.output.T
            
        ax.tick_params(top=False, bottom=True, right = False,
                       labeltop=False, labelbottom=True, labelright=False)

        ax.xaxis.set_label_position('bottom')
        ax.set_xscale('linear')
        ax.set_xlabel('delay')
        ax.set_ylabel(UNIT)
        
        #axr = ax.twiny()
        #axr.plot(ref.values, i_x, c=REFERENCE_COLOR)
        #axr.axis('off')
    
    def plot_deltaI(self, key='IFull', ax_i = 0):
        #print('is deltaI')
        self.recreate_subplots(1, 1)
        self.output = pd.DataFrame()
        base = self.DV.aggregation_params['ref_grp']
        if base:
            ref = self.DV.reference
            
        temp = self.DV.get_intensity(key)
        
        super_group_d = []
        #Add supergroups
        deltaI_choice = self.group_params['deltaI_choice']
        deltaI_dict = self.group_params['deltaI_dict']
        for supergroup in deltaI_choice:
            grps = deltaI_dict[supergroup]
            mean = xg.dv.mean_diff_i['IFull'].loc[grps].mean(axis=0)
            mean.name = supergroup
            super_group_d.append(mean)
        
        selection = self.plot_params['q_index']  
        
        i_x = self.DV.i_x
        left = 0
        right = len(i_x)
        x_axis = i_x
        
        if selection:
            left =  np.where(i_x >= selection[0])[0].min()
            right = np.where(i_x <= selection[1])[0].max()+1
            x_axis = x_axis[left:right]
            if base:
                ref = ref[left:right]   
                
        if self.plot_params['use_group']:
            
            delay_index = np.argwhere(np.array(self.DV.keywords)=='delay')[0][0]
            delays = np.sort(np.array([elem[delay_index] for elem in temp.index.values]))            
            
            y_axes = [temp.loc[grp][list(range(left, right))] for grp in temp.index.values]        
            min_val = 0
            max_val = len(y_axes)

            for i,y_axis in enumerate(y_axes):
                label = y_axis.name[delay_index]
                color = ((i - min_val)/(max_val - min_val))
                self.plot_sequence(x_axis, y_axis, label=label, color=mpl.colormaps[LINEAR_COLORMAP](color), ax_i=ax_i)
        
        y_super = [elem[list(range(left, right))] for elem in super_group_d] #[temp.loc[grp][list(range(left, right))] for grp in temp.index.values]
        
        for super_plot in y_super:
            self.plot_sequence(x_axis, super_plot, ax_i=ax_i)
        
        if base:
            self.plot_sequence(x_axis, ref/100.0, label='Reference', color=REFERENCE_COLOR, ax_i=ax_i)
        
        #pos = self.axs[0].get_position()
        
        self.axs[0].set(xlabel=UNIT, ylabel= 'deltaI')  
        self.axs[0].legend(loc='upper center', ncol=5, prop={'size': 6})
 
    def plot_delay(self, key='IFull', ax_i = 0):
        #print('is delay')
        self.recreate_subplots(1, 1)
        self.output = pd.DataFrame()
        base = self.DV.aggregation_params['ref_grp']
        temp = self.DV.get_intensity(key)
        delays = (temp.index.to_frame(index=False)['delay'])
        temp = temp.T

        delay_choice = self.group_params['delay_choice']
        delay_dict = self.group_params['delay_dict']
       
        selection = self.plot_params['q_index']
        i_x = self.DV.i_x
    
        if self.plot_params['use_q']:
            x_axis = delays
            left = 0
            right = 1
            if selection:
                left =  np.where(i_x >= selection[0])[0].min()
                right = np.where(i_x <= selection[1])[0].max()+1
            
            #temp = temp.iloc[list(range(left, right))] 
            y_axes = [temp.iloc[col] for col in range(left, right)]
            #i_x = i_x[left:right]
            
            min_val = i_x[left]
            max_val = i_x[right-1]
            #prevent division by zero
            offset = .01*(max_val-min_val)
            
            for y_axis in y_axes:
                label=y_axis.name
                color = ((offset+ label - min_val)/(offset+ max_val - min_val))
                self.plot_sequence(x_axis, y_axis, label=label, color=mpl.colormaps[LINEAR_COLORMAP](color), ax_i=ax_i)          
        
        super_group_q = []
        for supergroup in delay_choice:
            grps = delay_dict[supergroup]
            l =  np.where(i_x >= grps[0])[0].min()
            r = np.where(i_x <= grps[1])[0].max()+1
            mean = temp.iloc[list(range(l, r))].mean(axis=0)
            mean.name = supergroup
            super_group_q.append(mean)
        
        for superplot in super_group_q:
            x_ax = superplot.reset_index()['delay']
            self.plot_sequence(x_ax, superplot, ax_i=ax_i)
        
        self.axs[0].set(xlabel='delay', ylabel= 'deltaI')
        if USE_LOG:
            self.axs[0].set_xscale('symlog', linthresh=LINEAR_THRESHOLD) # how delay plot is plotted 
        #self.axs[0].set_xscale('log', linthresh=1000e-9) # how delay plot is plotted 
        self.axs[0].legend(loc='lower center', ncol=5, prop={'size': 6})
  
    def plot_sequence(self, x_axis, y_axis, label = None, ax_i=0, color = None):

        x_vals = x_axis.values
        x_label = x_axis.name
        y_vals = y_axis.values
        y_label = y_axis.name        
        #color = None
        ax = self.axs[ax_i]

        #label = self.DV.group_tostring(y_label)
        if label is None:
            label = y_label
        ax.plot(x_vals, y_vals, c=color, label = label)
            
        self.output = pd.concat([self.output, pd.Series(y_vals, index=x_vals, name=label)], axis=1)         
    
    def plot_stability(self, do_center=None, key='IFull', ax_i = 0):
        self.recreate_subplots(1, 1)
        temp = self.DV.properties
        ref_grp = self.DV.aggregation_params['ref_grp']
        if USE_SKIP and ref_grp and len(ref_grp) > 0:
            temp = self.DV.do_skip_difference()
            #temp = temp.loc[~(temp == 0).all(axis=1)]
        
        group_properties = self.DV.group_categories
        step = self.plot_params['step']
        stable_grp = self.plot_params['sample']

        #length = temp.groupby(group_properties).size().min()
        
        collect = pd.DataFrame()
        droplist = [x for x in temp.index.names if x not in self.DV.group_categories]
        
        length = len(temp.reset_index(droplist).loc[stable_grp][key])
        
        if do_center is None:
            do_center = self.plot_params['center']
        
        if do_center:
            selection = (temp.reset_index(droplist).loc[stable_grp]['Common'])[['c_x','c_y']].apply(pd.to_numeric)
            for i in range(step, length, step):
                column = (selection[i-step:i].mean(axis=0)).rename('{}-{}'.format(i-step, i))
                collect = collect.append(column)
            collect = collect.reset_index()#.reset_index().set_index('index')
            collect['index'] = pd.Categorical(collect['index'])
            collect.plot.scatter(x='c_x', y='c_y', c = 'index', cmap=LINEAR_COLORMAP)
            self.output = collect
            
        else:
            selection = temp.reset_index(droplist).loc[stable_grp][key] 

            for i in range(step, length+1, step):
                column = (selection[i-step:i].mean(axis=0)).rename('{}-{}'.format(i-step, i))
                collect = collect.append(column)
        
            collect.T.plot(cmap=LINEAR_COLORMAP, ax = self.axs[ax_i])
            self.output = collect.T
        

# Live Updateing Class

In [21]:

class ImageFilter(DefaultFilter):
    
    def __init__(self, prefix, **kwargs):
        self.allowed_extensions = '.tif'
        self.allowed_prefix = prefix
        super().__init__(**kwargs)
    def __call__(self, change: Change, path: str) -> bool:
        filename = os.path.basename(path)
        return (
            super().__call__(change, path) and 
            filename.startswith(self.allowed_prefix) and
            filename.endswith(self.allowed_extensions)
        )

#for changes in watch('./sharp2', watch_filter=ImageFilter('Ex02_Sa05_Sc02')):
    #print (changes)

In [22]:
class OTFAnalysis:
    
    def __init__(self, dsm, dv, logger=None):
        self.dsm = dsm
        self.dv = dv
        self.task = None
        self.static_task = None
        self.live_task = None
        self.watch_directories = None
        self.debounce = 500
        self.timeout = 20000
    
    def static_integrate(self):
        loop = asyncio.get_event_loop()
        asyncio.run_coroutine_threadsafe(self.dsm.static_integrate(), loop)       

    async def process_files(self, files):
        await self.dsm.live_calculate(files)
        
    async def main(self, filenames, tasks):

        async for changes in awatch(self.watch_directories, debounce = self.debounce, 
                                    rust_timeout = self.timeout,
                                    yield_on_timeout = True,
                                    watch_filter=ImageFilter(prefix = self.dsm.general_params['prefix'])):

            new_files = set()

            for change in changes:

                if (change[0] == Change.modified) or (change[0] == Change.added):
                    new_files.add(os.path.basename(change[1]))

            diff = new_files - filenames
            
            if len(diff) > 0:
                filenames.update(diff)
                t = asyncio.get_event_loop().create_task(self.process_files(diff))
                tasks.append(t)
                
            tasks = [task for task in tasks if not task.done()]
                
        print('watch task yielded, something may have went wrong')

    async def monitor(self):
        flag = True
        filenames = set()
        tasks = []
        try:
            await self.main(filenames, tasks)

            tasks = [t for t in tasks if not t.done()]

            print('tasks left: ', len(tasks))
            if len(tasks) > 0:
                print('Maybe cancelled but tasks remain?')
                while len(tasks) > 0:
                    tasks = [t for t in tasks if not t.done()]
                    asyncio.sleep(1)
            else:
                print('Cancellation?...live watch ending')
                self.task = None

        except asyncio.CancelledError:
            print('Watch Task Cancelled!')
                
    def begin_watch(self, directories=None, debounce = 500, timeout = 20000):
        if self.task is not None:
            print('''A watch task appears to already be running.  
                  If you want to change the current task, 
                  please stop the task first, then start 
                  it again with new parameters.''')
            return
        if directories == None:
            directories = self.dsm.general_params['src_dir']
        print('watch directory: {}'.format(directories))
        self.watch_directories = directories
        self.debounce = debounce
        self.timeout = timeout
        
        try:
            #loop = asyncio.get_event_loop()
            self.task = asyncio.get_event_loop().create_task(self.monitor())
            #self.task = asyncio.run_coroutine_threadsafe(self.monitor(), loop)
        except asyncio.CancelledError:
            print('Watch Task Cancelled!')

        
    def end_current_watch(self):
        if (self.task is not None):
            self.task.cancel()
            self.task = None
        else:
            print('No watch task running currently.')    
    def live_tasks(self, full_task):
        #add other params
        if (self.task is None):
            self.begin_watch()
        #pick a self.dv.mean_difference/... func and pass it on
        loop = asyncio.get_event_loop()
        self.live_task = loop.create_task(self.live_task_update(full_task))

        
    async def live_task_update(self, full_task):
        
        while (True):
            #print('in while')
            _ = await asyncio.get_event_loop().run_in_executor(None, full_task)   
            #print('will sleep')
            await asyncio.sleep(20)
            #print('will loop')
        #print('out of loop')
            
    def end_live_tasks(self):
        #self.end_current_watch()
        
        if (self.live_task is not None):
            self.live_task.cancel()
            self.live_task = None
        else:
            print('No live task running currently.')    
    

In [23]:
def stop_all_async():
    for e in asyncio.all_tasks(asyncio.get_event_loop()):
        e.cancel()

In [24]:
def list_all_async():
    print(asyncio.all_tasks(asyncio.get_event_loop()))

In [25]:
def print_tasks_running():
    task_set = asyncio.all_tasks(asyncio.get_event_loop())
    live_tasks = 0
    live_integ = 0
    for task in task_set:
        if str(task).find('monitor') > 0:
            live_integ+=1
        if str(task).find('live_task_update') > 0:
            live_tasks+=1
    print('Live Integration instances: {}, Live Tasks instances: {}'.format(live_integ, live_tasks))
        

# UI/Interaction Class

- 4 sections for each of the prior classes?
1. A panel for live integration controls (starting watchtask button + its parameters)
2. A panel for DSM parameters, plus a button for just the static integration
3. A panel for where plots are displayed
4. Buttons for creating the parsed instructions -> output stored locally for the plotting utility?

The function below likely disables certain notebook logging functionality, but you won't see the log being outputted below the last run cell/widget cell anymore

In [26]:
def VLabel(widget, label):
    return VBox([Label(value=label), widget], layout=Layout(width='auto', height='auto'))

In [43]:
class XRDGUI:
    
    def __init__(self):
        
        self.children = []
        
        self.plot_params = {}
        self.super_params = {'super_d': [], 'super_q': []}
        self.calib_params = {'center_aasign': False, 'center_dir': None, 'integ_dir': None}
        self.auto_params = {'all': False, 'mode': 'deltaI'}
        
        self.dsm_widgets = {}
        self.dv_widgets = {}
        self.dv_widgets['automation'] = {}
        self.dv_widgets['super'] = {}
        
        logger.info('XRDGUI begin construction')
        logger.info('initialize data...')
        self.initialize_data()

        logger.info('make plot widgets...')
        self.children.append(self.child_plot())
        logger.info('make panel tabs...')
        self.children.append(self.make_panel_tabs())
        self.link_dsm()
        
        logger.info('assembling full widget...')
        self.initialize_grid()
        logger.info('XRD GUI construction complete!')
        self.plot_params['graph_out'].clear_output(wait=True)
        self.plot_params['calib_out'].clear_output(wait=True)
        plt.close('calibration')
        plt.close('Interactive Plot')
            
    def initialize_data(self):
        self.dsm = DataScanManager()
        self.update_dsm(default=True)
        
        self.dv = DataViewer3(self.dsm)

        self.pm = PlotManager(self.dv)
        
        self.otf = OTFAnalysis(self.dsm, self.dv)

    def full_task(self):

        plot = self.auto_params['mode']
        agg = self.dv.aggregation_params['aggregator']
        
        targets = self.dv.parse_names_string(list(self.dv_widgets['automation']['active'].value))
        d_index = self.pm.plot_params['q_index']
        dv_task = None
        
        if agg == 'avg':
            dv_task = self.dv.mean_difference
        elif agg == 'nth':
            dv_task = self.dv.get_nth
   
        plot_task = None
        logger.info("plot_type is: {}".format(plot))
        if plot == 'heatmap':
            plot_task = self.pm.plot_heatmap #(self, ax_i, base, alternates, selection = None, v_range = .12)
        elif plot == 'deltaI':
            plot_task = self.pm.plot_deltaI
        elif plot == 'delay':
            plot_task = self.pm.plot_delay
        elif plot == 'stability':
            plot_task = self.pm.plot_stability

        self.dv.group_mode()
        self.dv.do_aggregation()
        #dv_task()
        plot_task(ax_i = 0)
        with self.plot_params['graph_out']:
            clear_output()
            plt.show()
            
        logger.info('full task complete')

    def update_dsm(self, default=True):
        if default:
           
            self.dsm.assign_general_params(src_dir = SRC_DIR, 
                                           dest_dir=DEST_DIR, 
                                           prefix = PREFIX, 
                                           keywords = KEYWORDS,
                                           mask_dir = MASK_DIR)
            self.dsm.assign_detector_params(
                                            distance = DETECTOR_DISTANCE,
                                            pixel_size = DETECTOR_PIXEL_SIZE,
                                            center = DETECTOR_CENTER,
                                            tilt_angle = DETECTOR_ANGLE,
                                            tilt_rotation = DETECTOR_ROTATION,
                                            wavelength = DETECTOR_WAVELENGTH
                                           )
            self.dsm.assign_integration_params(method=INTEGRATOR_METHOD)
            #self.dsm.preload_mask('Z:\\burakg\\2022\\06_Sharp_TRXRD_BIVO\\mask_2021_dec.tif')
            self.dsm.assign_center_params('hough', parameters = {
                                          'initial_threshold': CENTER_H_THRESH,
                                          'use_rounding': CENTER_H_ROUND,
                                          'minRadius': CENTER_H_MIN_RADIUS,
                                          'maxRadius': CENTER_H_MAX_RADIUS,
                                          'dp': 1,
                                          'circle_count': CENTER_H_COUNT,
                                          'mask_correlation': CENTER_H_MASK
                                          })
            
            #self.dsm.preload_mask(file=None)
            
        else:
            pass
    
    def initialize_grid(self):
        
        self.grid = GridBox(children = self.children,
                            layout=Layout(
                                          width='100%',
                                          grid_template_rows=' 100%',
                                          grid_template_columns='30% 70%',
                                          grid_template_areas=
                                          '''
                                          "panel plot"
                                          '''
                                         )
                            )
    
    #####################LINKING SECTION#####################################
    def link_dsm(self):
        #Link General
              
        def callback_build(widget, dictionary, key, func=None, index=None, names = 'value'):
            if type(widget) == list:
                #for index, widg in enumerate(widget):
                def callback_0(change):
                    logger.info("{}: {}".format(key, change['new']))
                    ##################!!!!!!!!!!!!!!NOTE: SO FAR NO PARAMETER USES A NEGATIVE VALUE, THUS THEIR PRESENCE IN RANGE VALUES INDICATES None is the desired value!!!!!!!!!!!!!###########
                    val = change['new']
                    if val < 0:
                        val = None
                    
                    if dictionary[key]:
                        dictionary[key][0] = val
                    else:
                        temp_val = [None, None]
                        temp_val[0] = val
                        dictionary[key] = temp_val
                        
                def callback_1(change):
                    logger.info("{}: {}".format(key, change['new']))
                    val = change['new']
                    if val < 0:
                        val = None
                    
                    if dictionary[key]:
                        dictionary[key][1] = val
                    else:
                        temp_val = [None, None]
                        temp_val[1] = val
                        dictionary[key] = temp_val
                
                widget[0].observe(callback_0, names=names)
                widget[1].observe(callback_1, names=names)
                return
                
            if func != None:
                def callback_(change):
                    logger.info("{}: {}".format(key, change['new']))
                    dictionary[key] = func(change['new'])           
            else:
                def callback_(change):
                    logger.info("{}: {}".format(key, change['new']))
                    dictionary[key] = change['new']

            widget.observe(callback_, names=names)
        
        def reverse_callback(widget, dictionary, key, func=None, index=None, names = 'value'):
            def setter(widget, names, value):
                if names == 'value':
                    widget.value = value
                elif names == 'selected_index':
                    widget.selected_index = value
                else:
                    logger.info('names has no match')
                    
            if type(widget) == list:
                #for index, widg in enumerate(widget):
                setter(widget[0], names, dictionary[key][0])
                setter(widget[1], names, dictionary[key][1])
                return
            
            if func != None:
                setter(widget, names, func(dictionary[key]))
            else:
                setter(widget, names, dictionary[key])
            
        #############################General Linking#######################################
        w_gen = self.dsm_widgets['general']
        
        callback_build(w_gen['src_dir'], self.dsm.general_params, 'src_dir')
        callback_build(w_gen['dest_dir'], self.dsm.general_params, 'dest_dir')
        callback_build(w_gen['prefix'], self.dsm.general_params, 'prefix')
        traitlets.directional_link((w_gen['prefix'], 'value'), (w_gen['scan_name'], 'value'))
        callback_build(w_gen['scan_name'], self.dsm.general_params, 'scan_name')        
        #callback_build(w_gen['keywords'], self.dsm.general_params, 'keywords', func=lambda x: x.split('\n'))
        def parse_callback(change):
            global PARTIAL_PARSE_STRING, KEYWORDS
            PARTIAL_PARSE_STRING = change['new']
            keys = PARTIAL_PARSE_STRING.split('{}')
            KEYWORDS = [word for word in keys if len(word) > 0]
            self.dv_widgets['grouper']['select'].options = KEYWORDS
            
        w_gen['parse_string'].observe(parse_callback, names='value')

        callback_build(w_gen['mask_dir'], self.dsm.general_params, 'mask_dir')
        ############################Detector Linking#########################################
        w_det = self.dsm_widgets['detector']
        
        callback_build(w_det['distance'], self.dsm.detector_params, 'distance')
        callback_build(w_det['pixel_size'], self.dsm.detector_params, 'pixel_size')
        callback_build(w_det['tilt_angle'], self.dsm.detector_params, 'tilt_angle')
        callback_build(w_det['tilt_rotation'], self.dsm.detector_params, 'tilt_rotation')
        callback_build(w_det['wavelength'], self.dsm.detector_params, 'wavelength')
        callback_build(w_det['detector'], self.dsm.detector_params, 'detector')
        callback_build(w_det['center'], self.dsm.detector_params, 'center')

        ############################Integration Linking########################################
        w_int = self.dsm_widgets['integrator']
        
        callback_build(w_int['npt'], self.dsm.integration_params, 'npt')
        callback_build(w_int['normalization_factor'], self.dsm.integration_params, 'normalization_factor')
        callback_build(w_int['radial_range'], self.dsm.integration_params, 'radial_range')
        callback_build(w_int['azimuth_range'], self.dsm.integration_params, 'azimuth_range')
        #callback_build(w_int['method'], self.dsm.integration_params, 'method')
        callback_build(w_int['calib_range'], self.dsm.calculation_params, 'norm_range')
        
        ####Calibration Button Callback, 'calibration'
        def on_integ_calibrate_click(b):
            self.dsm.preload_mask()
            self.dsm.calibrate_normalization(target = self.calib_params['integ_dir'], mean_region = None, autoassign=False)
            
            with self.plot_params['calib_out']:
                plt.show()
                clear_output(wait=True)     
                
        w_int['calibration'].on_click(on_integ_calibrate_click)
        
        ####Directory callback...?, 'directory'
        callback_build(w_int['directory'], self.calib_params, 'integ_dir')
        '''
        def integ_dir_update(chooser):
            self.calib_params['integ_dir'] = chooser.value
        w_int['directory'].register_callback(integ_dir_update)
        '''
        ################################Centering Linking########################################
        w_cir = self.dsm_widgets['circle']
        
        callback_build(w_cir['aasign'], self.calib_params, 'center_aasign')
        
        callback_build(w_cir['method'], self.dsm.center_params, 'method', func=lambda x: list(('hough', 'correlate'))[x], names='selected_index')
        
        callback_build(w_cir['minRadius'], self.dsm.center_params, 'minRadius')
        callback_build(w_cir['maxRadius'], self.dsm.center_params, 'maxRadius')        

        #callback_build(w_cir['radius_range'], self.dsm.center_params, 'minRadius', func=lambda x: x[0])
        #callback_build(w_cir['radius_range'], self.dsm.center_params, 'maxRadius', func=lambda x: x[1])
        
        callback_build(w_cir['dp'], self.dsm.center_params, 'dp')
        callback_build(w_cir['initial_threshold'], self.dsm.center_params, 'initial_threshold')
        callback_build(w_cir['circle_count'], self.dsm.center_params, 'circle_count')
        callback_build(w_cir['param1'], self.dsm.center_params, 'param1')
        callback_build(w_cir['param2'], self.dsm.center_params, 'param2')
        callback_build(w_cir['use_rounding'], self.dsm.center_params, 'use_rounding')
        callback_build(w_cir['mask_correlation'], self.dsm.center_params, 'mask_correlation')
        callback_build(w_cir['rescaler'], self.dsm.center_params, 'rescaler')
        
        ###Calibration Button Callback, 'calibration'
        def on_center_calibrate_click(b):
            self.dsm.preload_mask(file=None)
            center = self.dsm.calibrate_circle(target = self.calib_params['center_dir'], rescaler = self.dsm.center_params['rescaler'], autoassign = False)
            if self.calib_params['center_aasign']:
                w_det['center'][0].value = center[0]
                w_det['center'][1].value = center[1]            
            
            with self.plot_params['calib_out']:
                #plt.clf()
                #display(plt.show())
                plt.show()
                clear_output(wait=True)    
        w_cir['calibration'].on_click(on_center_calibrate_click)            
        ####Directory callback...?, 'directory'
        callback_build(w_cir['directory'], self.calib_params, 'center_dir')
        '''
        def center_dir_update(chooser):
            self.calib_params['center_dir'] = chooser.value
        w_cir['directory'].register_callback(center_dir_update)
        '''
        ################################## Link Integration #####################################
        callback_build(self.dsm_widgets['integration']['centering'], self.dsm.calculation_params, 'track_center')
        
        def static_integrate_trigger(trigger):
            logger.info('callback')
            self.otf.static_integrate()
        self.dsm_widgets['integration']['static'].on_click(static_integrate_trigger)
        
        def live_integrate_trigger(trigger): 
            #asyncio.get_event_loop().create_task(test_async())
            self.otf.begin_watch()
            print('live_integration started!')
        self.dsm_widgets['integration']['live'].on_click(live_integrate_trigger)
        
        def live_integrate_stop(trigger):
            self.otf.end_current_watch()
            print('live_integration ended!')
            
        self.dsm_widgets['integration']['stop'].on_click(live_integrate_stop)
        
        ################################## Data I/O ################################
        
        w_dsm_save = self.dsm_widgets['saver']
        
        def save_dsm_trigger(trigger):
            name_val = w_dsm_save['name'].value
            name = None
            if len(name_val) > 0:
                name = name_val
            
            self.dsm.savedata(name=name)
            logger.info('h5 and json saved!')
        
        traitlets.directional_link((w_gen['scan_name'], 'value'), (w_dsm_save['name'], 'value'))
        w_dsm_save['save'].on_click(save_dsm_trigger)
        
        def load_dsm_trigger(trigger):
            name_val = w_dsm_save['name'].value
            if len(name_val) > 0:
                self.dsm.loaddata(name=name_val)
            logger.info('New Values Loaded')
    
            #Set all ui elements by their loaded dsm counterparts
            reverse_callback(w_gen['src_dir'], self.dsm.general_params, 'src_dir')
            reverse_callback(w_gen['dest_dir'], self.dsm.general_params, 'dest_dir')
            reverse_callback(w_gen['prefix'], self.dsm.general_params, 'prefix')
            #reverse_callback(w_gen['extension'], self.dsm.general_params, 'extension')
            reverse_callback(w_gen['scan_name'], self.dsm.general_params, 'scan_name')
            '''
            def mask_dir_update(chooser):
                self.dsm.general_params['mask_dir'] = chooser.value

            w_gen['mask_dir'].register_callback(mask_dir_update)
            '''
            reverse_callback(w_gen['mask_dir'], self.dsm.general_params, 'mask_dir')
            ############################Detector#########################################
            w_det = self.dsm_widgets['detector']

            reverse_callback(w_det['distance'], self.dsm.detector_params, 'distance')
            reverse_callback(w_det['pixel_size'], self.dsm.detector_params, 'pixel_size')
            reverse_callback(w_det['tilt_angle'], self.dsm.detector_params, 'tilt_angle')
            reverse_callback(w_det['wavelength'], self.dsm.detector_params, 'wavelength')
            reverse_callback(w_det['detector'], self.dsm.detector_params, 'detector')
            reverse_callback(w_det['center'], self.dsm.detector_params, 'center')

            ############################Integration########################################
            w_int = self.dsm_widgets['integrator']

            reverse_callback(w_int['npt'], self.dsm.integration_params, 'npt')
            reverse_callback(w_int['normalization_factor'], self.dsm.integration_params, 'normalization_factor')
            if self.dsm.integration_params['radial_range'] == None:
                w_int['radial_range'] = [-1, -1]

            if self.dsm.integration_params['azimuth_range'] == None:
                w_int['azimuth_range'] = [-1, -1]
     
            reverse_callback(w_int['calib_range'], self.dsm.calculation_params, 'norm_range')

            #Redundant, load doesn't affect UI object
            #reverse_callback(w_int['directory'], self.calib_params, 'integ_dir')

            ################################Centering########################################
            w_cir = self.dsm_widgets['circle']

            reverse_callback(w_cir['aasign'], self.calib_params, 'center_aasign')

            reverse_callback(w_cir['method'], self.dsm.center_params, 'method', func=lambda x: ['hough', 'correlate'].index(x), names='selected_index')

            reverse_callback(w_cir['minRadius'], self.dsm.center_params, 'minRadius')
            reverse_callback(w_cir['maxRadius'], self.dsm.center_params, 'maxRadius')

            reverse_callback(w_cir['dp'], self.dsm.center_params, 'dp')
            reverse_callback(w_cir['initial_threshold'], self.dsm.center_params, 'initial_threshold')
            reverse_callback(w_cir['circle_count'], self.dsm.center_params, 'circle_count')
            reverse_callback(w_cir['param1'], self.dsm.center_params, 'param1')
            reverse_callback(w_cir['param2'], self.dsm.center_params, 'param2')
            reverse_callback(w_cir['use_rounding'], self.dsm.center_params, 'use_rounding')
            reverse_callback(w_cir['mask_correlation'], self.dsm.center_params, 'mask_correlation')
            reverse_callback(w_cir['rescaler'], self.dsm.center_params, 'rescaler')
            
            #Redundant, as loading doesn't affect the UI object
            #reverse_callback(w_cir['directory'], self.calib_params, 'center_dir')
        
        #Finally create callback of above method
        w_dsm_save['load'].on_click(load_dsm_trigger)        
    ###############################################################
    ####################LINK DATAVIEWER#############################
        
        #w_scan = self.dv_widgets['scan']
        
        #TODO: Implement Scan Widgets
        w_grp = self.dv_widgets['grouper']
        ##########################Grouper Stage#######################################
        def grouper_trigger(trigger):

            async def grouper_callback():
                
                val = list(w_grp['select'].value)
                logger.info('grouping values: {}'.format(val))
                
                if len(val) > 0:
                    #async with lock:
                    await self.dv.pull_data()
                    self.dv.group_mode(categories = val)
                    #TODO: Populate Group selectors with group values
                    w_auto['default'].options = self.dv.get_group_names()
                    w_auto['active'].options = self.dv.get_group_names()            
                    w_auto['stable_grp'].options = self.dv.get_group_names()
                    self.dv_widgets['super']['select_d'].options = self.dv.get_group_names()
                     
            asyncio.get_event_loop().create_task(grouper_callback())        
        w_grp['trigger'].on_click(grouper_trigger)
        
        
        w_auto = self.dv_widgets['automation']        
        
        ###############################Selection Stage###############################
        def select_all_trigger(trigger):
            logger.info('select all')
            self.auto_params['all'] = True
            w_auto['active'].value = list(w_auto['active'].options)
            self.dv.aggregation_params['groups'] = None

        def select_none_trigger(trigger):
            logger.info('clear all')
            self.auto_params['all'] = False
            w_auto['active'].value = []
            self.dv.aggregation_params['groups'] = []
        
        w_auto['all'].on_click(select_all_trigger)
        w_auto['none'].on_click(select_none_trigger)
        
        callback_build(w_auto['default'], self.dv.aggregation_params, 'ref_grp', func = self.dv.parse_names_string)
        callback_build(w_auto['active'], self.dv.aggregation_params, 'groups', func = self.dv.parse_names_string)
        
        #######################Automatic plot stages##################################
        def automation_select(change):
            self.auto_params['mode'] = ['deltaI', 'heatmap', 'delay', 'stability'][change['new']] 
            logger.info('selected mode: {}'.format(self.auto_params['mode']))
            #logger.info("{}: {}".format('automation mode: ', self.auto_params['mode']))
        
        w_auto['mode'].observe(automation_select, names='selected_index')
        
        callback_build(w_auto['di_agg'], self.dv.aggregation_params, 'aggregator')
        callback_build(w_auto['di_nth'], self.dv.aggregation_params, 'n_range')
        callback_build(w_auto['hm_scale'], self.pm.plot_params, 'v_range')
        callback_build(w_auto['d_index'], self.pm.plot_params, 'q_index')
        callback_build(w_auto['stable_grp'], self.pm.plot_params, 'sample', func = self.dv.parse_names_string)
        callback_build(w_auto['step'], self.pm.plot_params, 'step')
        callback_build(w_auto['center'], self.pm.plot_params, 'center')
        
        def start_task_trigger(trigger):
            logger.info('Task Started')
            self.full_task()
        
        w_auto['start'].on_click(start_task_trigger)

        def save_plot_trigger(trigger):
            w_dv_save = self.dv_widgets['saving']
            name = w_dv_save['name'].value
            dest_dir = self.dsm.general_params['dest_dir']
            plt.savefig(os.path.join(dest_dir, name+'_plot.png'))
            self.pm.output.to_csv(os.path.join(dest_dir, name+'_plot.csv'))
        
        w_auto['plot'].on_click(save_plot_trigger)
        
        callback_build(w_auto['use_group'], self.pm.plot_params, 'use_group')
        callback_build(w_auto['use_q'], self.pm.plot_params, 'use_q')

        ###########################DATAVIEWER SAVER#############################
        w_dv_save = self.dv_widgets['saving']
        
        def save_task_trigger(trigger):
            #In case a user is confident they don't need to plot, the aggregated values may not exist yet and must be created
            self.dv.group_mode()
            self.dv.do_aggregation()

            name = w_dv_save['name'].value
            self.dv.savedata(name=name)

            dest_dir = self.dsm.general_params['dest_dir']        
            
            plt.savefig(os.path.join(dest_dir, name+'_plot.png'))
            
            logger.info('Saved!')
        
        traitlets.directional_link((w_dsm_save['name'], 'value'), (w_dv_save['name'], 'value'))
        traitlets.directional_link((w_dv_save['name'], 'value'), (w_dsm_save['name'], 'value'))
        
        w_dv_save['save'].on_click(save_task_trigger)

    #################################SUPER SELECT######################################
    #self.dv_widgets['super'].update({'select_d': w_select_grps_d, 'super_d': w_super_grps_d, 'create_d': w_create_grp_d, 'clear_d': w_clear_grp_d})
        w_super = self.dv_widgets['super']
        
        ###Delta I group select
        #callback_build(w_super['select_d'], self.super_params, 'super_d', func = self.dv.parse_names_string)
        
        #Changes the selection value to that of the selected group
        def restore_callback_d(names):
            if len(names) > 0:
                w_super['select_d'].value = [self.dv.group_tostring(x) for x in self.pm.group_params['deltaI_dict'][names[0]]]
            else:
                 w_super['select_d'].value = []
            return names
        
        callback_build(w_super['super_d'], self.pm.group_params, 'deltaI_choice', func = restore_callback_d)
        
        def update_supergroups_d():
            w_super['super_d'].options = list(self.pm.group_params['deltaI_dict'].keys())
        
        def make_super_trigger_d(trigger):
            logger.info('create supergroup')
            self.pm.group_params['deltaI_dict'][w_super['name_d'].value] = self.dv.parse_names_string(w_super['select_d'].value)
            update_supergroups_d()
               
        def clear_super_trigger_d(trigger):
            logger.info('delete supergroup')
            for elem in w_super['super_d'].value:
                xg.pm.group_params['deltaI_dict'].pop(elem)
                #w_super['select_d'].value = self.dv.group_tostring(self.pm.group_params['delay_dict'][])
            update_supergroups_d()
            #self.pm.group_params['delay_dict'][w_super['name_d'].value]
            #w_super['super_d'].options = w_super['name_d'].value
            
        w_super['create_d'].on_click(make_super_trigger_d)
        w_super['clear_d'].on_click(clear_super_trigger_d)
    
        ###Delay Q Range
        
        #selection of super
        #callback_build(w_super['select_q'], self.super_params, 'super_q', func = self.dv.parse_names_string)
        
        #Changes the selection value to that of the selected group
        def restore_callback_q(names):
            if len(names) > 0:
                val =  [x for x in self.pm.group_params['delay_dict'][names[0]]]
                w_super['select_q'][0].value = val[0]
                w_super['select_q'][1].value = val[1]
                
            return names
        
        callback_build(w_super['super_q'], self.pm.group_params, 'delay_choice', func = restore_callback_q)
        
        def update_supergroups_q():
            w_super['super_q'].options = list(self.pm.group_params['delay_dict'].keys())
        
        def make_super_trigger_q(trigger):
            logger.info('create supergroup')
            self.pm.group_params['delay_dict'][w_super['name_q'].value] = [w_super['select_q'][0].value, w_super['select_q'][1].value]
            update_supergroups_q()
               
        def clear_super_trigger_q(trigger):
            logger.info('delete supergroup')
            for elem in w_super['super_q'].value:
                xg.pm.group_params['delay_dict'].pop(elem)
                #w_super['select_d'].value = self.dv.group_tostring(self.pm.group_params['delay_dict'][])
            update_supergroups_q()
            #self.pm.group_params['delay_dict'][w_super['name_d'].value]
            #w_super['super_d'].options = w_super['name_d'].value
            
        w_super['create_q'].on_click(make_super_trigger_q)
        w_super['clear_q'].on_click(clear_super_trigger_q)
    
    ##################################################
    ##################################################
    ##################################################
    
    ##################PLOT SECTION####################
    def child_plot(self, region='plot'):
        #from https://kapernikov.com/ipywidgets-with-matplotlib/
        vb_layout = Layout(min_height = '1000px', min_width='600px')
        
        
        self.plot_params['graph_out'] = widgets.Output(clear_output=True, wait=True)
        self.plot_params['calib_out'] = widgets.Output(clear_output=True, wait=True)

        self.plot_params['plot'] = VBox(children=[Label(value='Plotting Menu', 
                                                        layout=Layout(justify_content="flex-start")), 
                                        self.plot_params['graph_out']], 
                                        layout=vb_layout)
        self.plot_params['calib'] = VBox(children=[Label(value='Calibration Menu', 
                                                         layout=Layout(justify_content="flex-start")), 
                                         self.plot_params['calib_out']], 
                                         layout=vb_layout)
        
        tabs = widgets.Tab(children = [self.plot_params['plot'], self.plot_params['calib']], 
                           layout=Layout(grid_area=region, width='100%'))
        tabs.set_title(0, 'Plotting') 
        tabs.set_title(1, 'Calibration')
        return tabs

    ###################### PANEL SECTION ##############################
    
    def make_panel_tabs(self, region='panel'):

        tabs = widgets.Tab(children = [self.make_dsm_component(), self.make_dataview_panel()], layout=Layout(grid_area = region))
        
        tabs.set_title(0, 'DSM')
        tabs.set_title(1, 'DataViewer')
        
        return tabs
    
    ######################   DSM SECTION   ########################   
    
    def make_dsm_component(self, region='dsm'):
        
        dsm_children = [self.child_DSM_general(), self.child_DSM_detector(), self.child_DSM_integrator(), self.child_DSM_circle(), self.child_DSM_saver()]
        accordion = widgets.Accordion(children = dsm_children)
        accordion.set_title(0, 'general')
        accordion.set_title(1, 'detector')
        accordion.set_title(2, 'integrator')
        accordion.set_title(3, 'centering')
        accordion.set_title(4, 'Saving')
        container = VBox([self.make_integration_controls(),
                          Label(value='DataScan Settings'), 
                          accordion], 
                         layout=Layout(width = 'auto', grid_area=region))
        
        return container
    
    ####General subsection####
    def child_DSM_general(self):
        params = self.dsm.general_params
        
        w_src_dir = Text(value = params['src_dir'],placeholder='e.g. /my/dir', style=dict(description_width='initial'), continuous_update=False)
        w_dest_dir = Text(value = params['dest_dir'],placeholder='e.g. /my/dir', style=dict(description_width='initial'), continuous_update=False)
        w_prefix = Text(value = params['prefix'],placeholder='e.g. SiO2_Scan01', style=dict(description_width='initial'), continuous_update=False)
        #w_keywords = widgets.Textarea(value = '\n'.join(params['keywords']), style=dict(description_width='initial'))
        #w_gen['parse_string']
        w_parse = Text(value = PARTIAL_PARSE_STRING, placeholder='e.g. fshw{}delay{}', style=dict(description_width='initial'), continuous_update=False)
        #w_extension = Text(value = params['extension'],placeholder='e.g. .tif', style=dict(description_width='initial'), continuous_update=False)
        
        w_scan_name = Text(value = params['scan_name'], placeholder='e.g. Sc01', style=dict(description_width='initial'), continuous_update=False)
        
        w_dir = Text(value = params['mask_dir'], placeholder='Z://', style=dict(description_width='initial'), continuous_update=False)

        main_container = VBox(children = [VLabel(w_src_dir, label = 'source directory'), VLabel(w_dest_dir, label = 'destination directory'), 
                                          VLabel(w_prefix, label = 'scan name prefix'), #VLabel(w_keywords, label = 'scan file keywords'), 
                                          VLabel(w_scan_name, label = 'Scan Name (convenient for post processing)'), 
                                          VLabel(w_parse, label = 'partial parse string'), 
                                          VLabel(w_dir, label = 'mask directory')])

        self.dsm_widgets['general'] = {'src_dir': w_src_dir, 'dest_dir': w_dest_dir, 'prefix': w_prefix, 
                                       'parse_string': w_parse, 'mask_dir': w_dir, 'scan_name': w_scan_name}
                                       #'separator': w_separator, 'keywords': w_keywords,
        
        return main_container
    
    ####Detector Subsection####
    def child_DSM_detector(self):
        
        params = self.dsm.detector_params
        w_distance = FloatText(value = params['distance'],placeholder='e.g. 200', style=dict(description_width='initial'), continuous_update=False)
        w_pixel_size = FloatText(value = params['pixel_size'],placeholder='e.g. 172', style=dict(description_width='initial'), continuous_update=False)
        w_cx = FloatText(value = params['center'][0],placeholder='e.g. 765', description = 'x:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_cy = FloatText(value = params['center'][1],placeholder='e.g. 60', description = 'y:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_center = VBox(children=[Label(value='Center (px)', layout=Layout(width='auto', height='auto')),
                        HBox(children=[w_cx, w_cy])])
        w_tilt_angle = FloatText(value = params['tilt_angle'],placeholder='e.g. 45.3', style=dict(description_width='initial'), continuous_update=False)
        w_tilt_rotation = FloatText(value = params['tilt_rotation'],placeholder='e.g. -.3', style=dict(description_width='initial'), continuous_update=False)
        w_wavelength = FloatText(value = params['wavelength'],placeholder='e.g. 1.06e-10', style=dict(description_width='initial'), continuous_update=False)
        w_detector = Text(value = params['detector'], placeholder='e.g. Pilatus2M (see pyFAI docs for others)', style=dict(description_width='initial'), continuous_update=False)
        
        self.dsm_widgets['detector'] = {'distance': w_distance, 'pixel_size': w_pixel_size, 'center': [w_cx, w_cy], 'tilt_angle': w_tilt_angle,
                                       'tilt_rotation': w_tilt_rotation, 'wavelength': w_wavelength, 'detector': w_detector} 
        
        main_container = VBox(children = [VLabel(w_distance, label='detector distance (mm)'), VLabel(w_pixel_size, label='pixel size (m)'), w_center, 
                                          VLabel(w_tilt_angle, label='tilt angle (deg)'), VLabel(w_tilt_rotation, label='tilt rotation (deg)'), 
                                          VLabel(w_wavelength, label='wavelength (m)'), VLabel(w_detector, label='detector name')])        
        
        return main_container
    
    ####Integrator Subsection####
    def child_DSM_integrator(self):
        params = self.dsm.integration_params
        w_npt = IntText(value = params['npt'],placeholder='e.g. 1000', description = 'number of points', style=dict(description_width='initial'), continuous_update=False)
        w_norm = FloatText(value = params['normalization_factor'],placeholder='e.g. 1.0, or set by calibration', description = 'Normalization', style=dict(description_width='initial'), continuous_update=False)
        rr = params['radial_range']
        if params['radial_range'] == None:
            rr=[-1, -1]
        w_rl = FloatText(value = rr[0], placeholder='e.g. <None> or 10', description = 'lower:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_rh = FloatText(value = rr[1], placeholder='e.g. <None> or 20', description = 'upper:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_radial = VBox(children=[Label(value='Radial Range (nm^-1)', layout=Layout(min_height='50px')), 
                                  HBox(children=[w_rl, w_rh])])
        
        ar = params['azimuth_range']
        if params['azimuth_range'] == None:
            ar=[-1, -1]
        w_al = FloatText(value = ar[0], placeholder='e.g. <None> or 10', description = 'lower:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_ah = FloatText(value = ar[1], placeholder='e.g. <None> or 20', description = 'upper:', layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_azimuth = VBox([Label(value='Azimuth Range (2theta)', layout=Layout(min_width='100px', min_height='50px')),
                          HBox(children=[w_al, w_ah])])
        
        #w_method = Text(value = params['method'], placeholder='e.g. csr, csr_ocl, etc.', style=dict(description_width='initial'), continuous_update=False)

        
        w_cal_l = FloatText(value = self.dsm.calculation_params['norm_range'][0], placeholder='0', description = 'lower:', 
                            layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_cal_h = FloatText(value = self.dsm.calculation_params['norm_range'][1], placeholder='10', description = 'upper:', 
                            layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_calib_range = VBox([Label(value='Q-Range: ', layout=Layout(min_width='100px', min_height='20px')),
                              HBox(children=[w_cal_l, w_cal_h])])
        
        w_calibration = widgets.Button(
                                        description='Do Calibration',
                                        disabled=False,
                                        button_style='info', 
                                        tooltip='See Calibration Plot'
                                      )
        
        w_dir = Text(placeholder='Z://', style=dict(description_width='initial'), continuous_update=False)
        #w_dir = FileChooser(select_desc='Select File',layout = Layout(width = "80%"))
        #w_dir.default_path = FILECHOOSER_DEFAULT_DIR
        
        self.dsm_widgets['integrator'] = {'npt': w_npt, 'normalization_factor': w_norm, 'radial_range': [w_rl, w_rh], 'azimuth_range': [w_al, w_ah],
                                          'calib_range': [w_cal_l, w_cal_h], 'calibration': w_calibration, 'directory':w_dir} 
                                          #'method': w_method, 
                                          
        
        main_container = VBox(children = [w_npt, w_norm, w_radial, w_azimuth, #VLabel(w_method, label='Integration Method (see PyFAI docs): '), 
                                          w_calib_range, w_dir, w_calibration]) 
        return main_container
    
    ##### Circle Subsection ####
    def child_DSM_circle(self):
        params = self.dsm.center_params
        method = params['method']
        
        w_aasign = widgets.Checkbox(
                                    value=False,
                                    description='Auto Assign Calibration',
                                    disabled=False,
                                    indent=False
                                    )
        
        w_rescale = widgets.FloatSlider(
                                        value=params['rescaler'],
                                        min=0.0,
                                        max=1.0,
                                        step=0.005,
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='.3f',
                                        )
        w_minRadius = IntText(value = params['minRadius'], placeholder='e.g. 300', style=dict(description_width='initial'), layout = Layout(width = "80%"), continuous_update=False)
        w_maxRadius = IntText(value = params['maxRadius'], placeholder='e.g. 500', style=dict(description_width='initial'),  layout = Layout(width = "80%"), continuous_update=False)
        
        w_dp = widgets.FloatSlider(
                                    value=params['dp'],
                                    min=1.0,
                                    max=5.0,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    )
        w_rs = widgets.FloatSlider(
                                    value=params['resizer'],
                                    min=0.1,
                                    max=1.0,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    )
        
        w_round = widgets.Checkbox(
                                   value=params['use_rounding'],
                                   disabled=False,
                                   indent=False
                                  ) 
        
        w_init_th = widgets.FloatSlider(
                                        value=0.0,
                                        min=-1.0,
                                        max=1.0,
                                        step=0.1,
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='.1f',
                                        )
        w_circle_count = widgets.IntSlider(
                                            value=params['circle_count'],
                                            min=1,
                                            max=10,
                                            step=1,
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d'
                                          )
        
        w_param1 = widgets.IntSlider(
                                    value=params['param1'],
                                    min=0,
                                    max=1000,
                                    step=1,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='d'
                                    )
        
        w_param2 = widgets.IntSlider(
                                    value=params['param2'],
                                    min=0,
                                    max=1000,
                                    step=1,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='d'
                                    )          
        
        hough_tab = VBox([HBox([VLabel(w_minRadius, label='Min Radius: '), VLabel(w_maxRadius, label='Max Radius: ')]), 
                          VLabel(w_dp, label='dp ("density of parameters"): '), 
                          VLabel(w_rs, label='Resizing Factor: '), VLabel(w_round, label='Use Rounding'), 
                          VLabel(w_init_th, label='Threshold Shift: '), VLabel(w_circle_count, label='Circle Count:'), 
                          VLabel(w_param1, label='Param1: '), VLabel(w_param2, label='Param2: ')])
        
        w_maskc = widgets.Checkbox(
                                    value=params['mask_correlation'],
                                    description='Apply Mask on Correlate',
                                    disabled=False,
                                    indent=False
                                  )
        
        correlation_tab = VBox([w_maskc])
        
        tabs = widgets.Tab(children = [hough_tab, correlation_tab])
        tabs.set_title(0, 'hough')
        tabs.set_title(1, 'correlate')
        w_calibration = widgets.Button(
                                        description='Do Calibration',
                                        disabled=False,
                                        button_style='info', 
                                        tooltip='See Calibration Plot'
                                      )               
        
        w_dir = Text(placeholder='Z://', style=dict(description_width='initial'), continuous_update=False)
        
        self.dsm_widgets['circle'] = {'aasign': w_aasign, 'rescaler': w_rescale, 'minRadius': w_minRadius, 'maxRadius': w_maxRadius, 
                                      'dp': w_dp, 'resizer': w_rs, 'initial_threshold': w_init_th, 'use_rounding': w_round,
                                      'circle_count': w_circle_count, 'param1': w_param1, 'param2': w_param2, 'mask_correlation': w_maskc,
                                      'method': tabs, 'calibration': w_calibration, 'directory': w_dir}        
        
        main_container = VBox([w_aasign, VLabel(w_rescale, label='factor applied to image, values then clamped'), tabs, w_dir, w_calibration])
        return main_container

    ###############################***************Saving and I/O section******************######################
    def child_DSM_saver(self):
        
        w_saver = widgets.Button(
                                description='Save Param. and Data',
                                disabled=False,
                                button_style='info', 
                                tooltip='Saves in pandas h5 compressed format',
                                layout=Layout(width='90%')
                                )
        w_loader = widgets.Button(
                                 description='Load Param. and Data',
                                 disabled=False,
                                 button_style='warning', 
                                 tooltip='Loads json and h5',
                                 layout=Layout(width='90%')
                                 )
        w_name = Text(placeholder='scan_name', style=dict(description_width='initial'), continuous_update=False)
        self.dsm_widgets['saver'] = {'save': w_saver, 'load':w_loader, 'name': w_name}
        
        return VBox([VLabel(w_name, 'Used to prefix saved data'), HBox([w_saver, w_loader])], layout=Layout(width='100%'))
        
    #############################***********INTEGRATION CONTROL SECTION***********###############################
    def make_integration_controls(self):
        w_center = widgets.Checkbox(
                                    value=False,
                                    description='Perform Centering Step',
                                    disabled=False,
                                    indent=False
                                    )
        w_static = widgets.Button(
                                  description='Integrate',
                                  disabled=False,
                                  button_style='info', 
                                  tooltip='integrates all of current directory, overwrites data in memory'
                                  )    
        w_live = widgets.Button(
                                description='Live',
                                disabled=False,
                                button_style='warning',
                                tooltip='integrates all new files in directory upon arrival'
                                )
        w_stop = widgets.Button(
                                description='Stop',
                                disabled=False,
                                button_style='danger',
                                tooltip='Stops live integration'
                                )
        w_buttons = HBox([w_static, w_live, w_stop])
        
        main_container = VBox([Label(value='Integration Controls'), w_center, w_buttons], layout = Layout(width='100%'))
        
        self.dsm_widgets['integration'] = {'centering': w_center, 'static': w_static, 'live' : w_live, 'stop': w_stop}
        
        return main_container
    
    #############################***********DATAVIEWER SECTION***********###############################
    
    def make_dataview_panel(self):
        #scan_manager = self.make_scan_manager()
        grouper = self.make_grouper()
        aggregator = self.make_aggregator()
        splitter = self.make_splitter()
        plotter = self.make_plotter()
        saver = self.make_saver()
        #accordion = Accordion([scan_manager, VBox([grouper, aggregator]), splitter, VBox([aggregator, plotter]), saver])
        accordion = Accordion([VBox([grouper, aggregator]), splitter, VBox([aggregator, plotter]), saver])
        #accordion.set_title(0, 'pull and load scans')
        accordion.set_title(0, 'group and aggregate')
        accordion.set_title(1, 'select and customize')
        accordion.set_title(2, 'plotting')
        accordion.set_title(3, 'Data Saving')
        container = VBox([Label(value='DataViewer Settings'), 
                          accordion], 
                         layout=Layout(width = 'auto'))
        
        return container
    
    
    ##########################################SCANMANAGER#######################################################
    def make_scan_manager(self):
        #Path Text Box
        
        w_load_dir = Text(value = None,placeholder='e.g. /my/dir', style=dict(description_width='initial'), continuous_update=False)
        w_load_name = Text(value = None,placeholder='e.g. Sc05 or Scan05', style=dict(description_width='initial'), continuous_update=False)
        #Load Button
        w_load = widgets.Button(
                                description='Load Datascan',
                                disabled=False,
                                button_style='info',
                                tooltip='Loads Datascan under given scanname'
                                )
        
        #Pull Button
        w_pull_name = Text(value = None, placeholder='e.g. Sc05 or Scan05', style=dict(description_width='initial'), continuous_update=False)
        
        w_pull = widgets.Button(
                                description='Pull Data',
                                disabled=False,
                                button_style='info',
                                tooltip='Pulls current Datascan data under the given scanname'
                                )
        
        w_selection = widgets.SelectMultiple(rows = 5, layout=Layout(width='flex'))
        
        w_delete = widgets.Button(
                                description='Delete Selected',
                                disabled=False,
                                button_style='danger',
                                tooltip='Deletes Scans from DataViewer'
                                )
        
        self.dv_widgets['scan'] = {'load_dir':w_load_dir, 'load_name':w_load_name, 'load_trigger':w_load, 'pull_name':w_pull_name, 'pull_trigger':w_pull,
                                   'select':w_selection, 'delete_trigger':w_delete}
        
        tabs = widgets.Tab([VBox([VLabel(w_pull_name, 'name of current integration scan'), w_pull]),
                            VBox([VLabel(w_load_dir, 'scan load directory'), VLabel(w_load_name, 'loaded scan name'), w_load])])
        
        tabs.set_title(0, 'pull')
        tabs.set_title(1, 'load')
        #Remember when linking for a callback to modify grouping parameters
        return VBox([tabs, 
                      VLabel(w_selection, 'Current Scans'), w_delete])
    
    ##########################GROUPER###################################
    def make_grouper(self):
         
        #fill in with grouping categories
        w_gselect = SelectMultiple(options= KEYWORDS, rows = 4, layout=Layout(width='flex'))
        #performs grouping step
        w_gtrigger = widgets.Button(
                                    description='Group!',
                                    disabled=False,
                                    button_style='warning', 
                                    tooltip='Groups by selected categories', 
                                    layout=Layout(width='flex')
                                   )         
        #Container for grouping components
        w_grouper = VBox([VLabel(w_gselect, 'Grouping Categories'), w_gtrigger], layout=Layout(width='100%'))
        
        self.dv_widgets['grouper'] = {'select':w_gselect, 'trigger': w_gtrigger}
        
        return w_grouper
    
    ############################AGGREGATOR#############################
    def make_aggregator(self):
        
        w_agg = widgets.Select(value = 'avg', options=['avg', 'nth'], rows=3, layout=Layout(max_width='50px'))
        
        w_n_l = IntText(value = self.dv.aggregation_params['n_range'][0], placeholder='0', description = 'lower:', 
                        layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_n_h = IntText(value = self.dv.aggregation_params['n_range'][1], placeholder='10', description = 'upper:', 
                        layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_nth = VBox([Label(value='N-index Range: ', layout=Layout(min_width='100px')),
                               HBox(children=[w_n_l, w_n_h])])
        
        w_frame = HBox([VBox([Label('Agg. Method'), w_agg], layout=Layout(min_width='100px')),
                        VBox([Label('nth if used'), w_nth])])

        self.dv_widgets['automation'].update({'di_agg': w_agg, 'di_nth': [w_n_l, w_n_h]})  
            
        return w_frame
        
    #####Default and Selected Group Selector
    def make_splitter(self):
        
        w_default_group = SelectMultiple(options= ['(fshw: -36, delay: -5e-10)'], rows=10, layout=Layout(max_width='175px'))
        w_active_groups = SelectMultiple(options= ['(fshw: -36, delay: -5e-10)'], rows=10, layout=Layout(max_width='175px'))
        
        w_groups = HBox([VLabel(w_default_group, 'Default Group'), VLabel(w_active_groups, 'Active Groups')])
        
        w_all = widgets.Button(
                              description='Select All',
                              disabled=False,
                              button_style='success', 
                              tooltip='select all active'
                              )
        
        w_none = widgets.Button(
                              description='Clear Selection',
                              disabled=False,
                              button_style='danger', 
                              tooltip='deselect all active'
                              )
        
        w_selection = HBox([w_all, w_none])
                            #, w_exec])
            
        self.dv_widgets['automation'].update({'default': w_default_group, 'active': w_active_groups, 'all': w_all, 'none': w_none}) 
            
        return VBox([w_groups, w_selection])
    
    ############################## SUPER GROUPERS ##########################################
    def make_super_d(self):

        w_name_d = Text(value = None, placeholder='<Group Name>', continuous_update=False)
        
        w_select_grps_d = SelectMultiple(rows=10, layout=Layout(max_width='250px'))
        w_super_grps_d = SelectMultiple(rows=10, layout=Layout(max_width='100px'))
        
        w_groups_d = HBox([VLabel(w_select_grps_d, 'Selected Groups'), VLabel(w_super_grps_d, 'Super Groups')])
        
        w_create_grp_d = widgets.Button(
                                        description='Make Group',
                                        disabled=False,
                                        button_style='info', 
                                        tooltip='creates supergroup'
                                       )
        
        w_clear_grp_d = widgets.Button(
                                       description='Delete Group',
                                       disabled=False,
                                       button_style='danger', 
                                       tooltip='deletes supergroup'
                                      )
        
        w_buttons_d = HBox([w_create_grp_d, w_clear_grp_d])
        w_contain_d = VBox([w_groups_d, w_name_d, w_buttons_d])
        
        self.dv_widgets['super'].update({'select_d': w_select_grps_d, 'super_d': w_super_grps_d, 'create_d': w_create_grp_d, 'clear_d': w_clear_grp_d,
                                         'name_d': w_name_d})
        
        return w_contain_d
    
    def make_super_q(self):
        #Q ranges groups for Delay Plot
        w_name_q = Text(value = None, placeholder='<Group Name>', continuous_update=False)
        
        w_q_l = FloatText(value = 0, placeholder='0', description = 'lower:', 
                            layout = Layout(width = "75%"), style=dict(description_width='initial'), continuous_update=False)
        w_q_h = FloatText(value = 1, placeholder='10', description = 'upper:', 
                            layout = Layout(width = "75%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_q_range = VBox([VLabel(w_q_l, 'Lower'), VLabel(w_q_h, 'Upper')], layout=Layout(max_width='175px'))
        
        w_super_grps_q = SelectMultiple(rows=10, layout=Layout(max_width='175px'))
        
        w_groups_q = HBox([VLabel(w_q_range, 'Q-Range Select'), VLabel(w_super_grps_q, 'Super Groups')])
        
        w_create_grp_q = widgets.Button(
                                        description='Make Group',
                                        disabled=False,
                                        button_style='info', 
                                        tooltip='creates supergroup'
                                       )
        
        w_clear_grp_q = widgets.Button(
                                       description='Delete Group',
                                       disabled=False,
                                       button_style='danger', 
                                       tooltip='deletes supergroup'
                                      )
        
        w_buttons_q = HBox([w_create_grp_q, w_clear_grp_q])
        w_contain_q = VBox([w_groups_q, w_name_q, w_buttons_q])
        
        self.dv_widgets['super'].update({'select_q': [w_q_l, w_q_h], 'super_q': w_super_grps_q, 'create_q': w_create_grp_q, 'clear_q': w_clear_grp_q,
                                         'name_q': w_name_q})
        
        return w_contain_q
        
    ############################PLOTTER##################################################
    def make_plotter(self):
        w_color_scale = FloatText(value = 0.12, placeholder='0.12', step=0.001, continuous_update=False)
        
        w_heatmap = VBox(children=[Label(value='HeatMap'), VLabel(w_color_scale, 'Max Intensity Value')])
        #Delay Widget
        
        w_q_l = FloatText(value = self.pm.plot_params['q_index'][0], placeholder='0', description = 'lower:', 
                            layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        w_q_h = FloatText(value = self.pm.plot_params['q_index'][1], placeholder='10', description = 'upper:', 
                            layout = Layout(width = "40%"), style=dict(description_width='initial'), continuous_update=False)
        
        w_qindex = VBox([Label(value='Q-Range: ', layout=Layout(min_width='100px')),
                               HBox(children=[w_q_l, w_q_h])])
        
        w_deltaI_use_group = widgets.Checkbox(
                                              value=False,
                                              disabled=False,
                                              indent=False,
                                              description='Include selection groups',
                                              layout=Layout(width='90%')
                                             )
        w_deltaI = VBox(children=[Label('Delta Intensity'), w_deltaI_use_group, self.make_super_d()])
        w_delay_use_q = widgets.Checkbox(
                                         value=False,
                                         disabled=False,
                                         indent=False,
                                         description='Include Q-Range (every q point)',
                                         layout=Layout(width='90%')
                                        )        
        w_delay = VBox(children=[Label('Delay Variation on fixed Q'), w_delay_use_q, self.make_super_q()])
        
        w_group = widgets.Select(options= ['(fshw: -36, delay: -5e-10)'],rows=10, layout=Layout(max_width='200px'))
        w_step = IntText(value = self.pm.plot_params['step'], placeholder='10', continuous_update=False, layout=Layout(width='90%'))
        w_center = widgets.Checkbox(
                                    value=False,
                                    disabled=False,
                                    indent=False,
                                    layout=Layout(width='90%')
                                    )
        
        w_stable = VBox(children=[Label('Ignores Aggregation, for checking scan consistency'), 
                                  HBox([w_group, 
                                        VBox([VLabel(w_step, 'Step Size'),
                                              VLabel(w_center, 'Track Center')
                                             ])
                                       ])
                                 ])
        
        w_automation = widgets.Tab(children = [w_deltaI, w_heatmap, w_delay, w_stable])
        
        w_automation.set_title(0, 'Delta I')
        w_automation.set_title(1, 'Heatmap')
        w_automation.set_title(2, 'Delay')
        w_automation.set_title(3, 'Stability')
        
        w_start = widgets.Button(
                                 description='Start',
                                 disabled=False,
                                 button_style='info', 
                                 tooltip='Plots Selected'
                                )
        

        w_plotsaver = widgets.Button(
                                     description='Save Plot',
                                     disabled=False,
                                     button_style='warning', 
                                     tooltip='saves the plot parameters'
                                     )
            
        global USE_LOG
        w_log_toggle = widgets.Checkbox(
                                        value=USE_LOG,
                                        disabled=False,
                                        indent=False,
                                        layout=Layout(width='90%')
                                        )
        
        def set_use_log(change):
            global USE_LOG
            USE_LOG = change['new']
        
        w_log_toggle.observe(set_use_log, names='value')
        
        w_starters = HBox([w_start, w_plotsaver]) #, w_startlive, w_stoplive])        
    
        self.dv_widgets['automation'].update({'hm_scale': w_color_scale, 'd_index': [w_q_l, w_q_h], 
                                              'stable_grp': w_group, 'step':w_step, 'center': w_center,
                                              'mode': w_automation, 'use_q': w_delay_use_q, 'use_group': w_deltaI_use_group,
                                              'start': w_start, 'plot': w_plotsaver}) 
                                              #'live': w_startlive, 'stop': w_stoplive})       
        
        return VBox([VLabel(w_log_toggle, 'USE LOG for heatmap, symlog for delay'), w_qindex, w_automation, w_starters])
    
    def make_saver(self):
        
        
        w_saver = widgets.Button(
                                 description='Save processed data',
                                 disabled=False,
                                 button_style='info', 
                                 tooltip='multiple .csv and .dtc'
                                )
        w_name = Text(placeholder='Prefix', style=dict(description_width='initial'), continuous_update=False)
        
        self.dv_widgets['saving'] = {'save':w_saver, 'name':w_name}
        
        return VBox([VLabel(w_name, 'File Prefix'), w_saver])
        

# Display Line

In [44]:
xg = XRDGUI()

INFO:__main__:XRDGUI begin construction
INFO:__main__:initialize data...
INFO:__main__:make plot widgets...
INFO:__main__:make panel tabs...
INFO:__main__:assembling full widget...
INFO:__main__:XRD GUI construction complete!


In [45]:
handler.clear_logs()

In [46]:
handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'), outputs=({'name': 'stdout', 'out…

In [47]:
display(xg.grid)

GridBox(children=(Tab(children=(VBox(children=(Label(value='Plotting Menu', layout=Layout(justify_content='fle…

# Additional Menus (to add)

In [56]:
def stop_async_trigger(trigger):
    stop_all_async()
def list_async_trigger(trigger):
    list_all_async()
def clear_logs_trigger(trigger):
    global handler
    handler.clear_logs()

In [57]:
additional_menu_elems = []
methods = [stop_async_trigger, list_async_trigger, clear_logs_trigger]
for method in methods:
    btn = Button(description=method.__name__)
    btn.on_click(method)
    additional_menu_elems.append(btn)

In [58]:
w_additional_menu = VBox(children=additional_menu_elems)

In [59]:
display(w_additional_menu)

In [55]:
USE_LOG

True